# 🐟 ViT-FishID: Semi-Supervised Fish Classification

**COMPLETE TRAINING PIPELINE WITH GOOGLE COLAB**

<a href="https://colab.research.google.com/github/cat-thomson/ViT-FishID/blob/main/ViT_FishID_Colab_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🎯 What This Notebook Does

This notebook implements a **complete semi-supervised learning pipeline** for fish species classification using:

**🤖 Vision Transformer (ViT)**: State-of-the-art transformer architecture for image classification
**📊 Semi-Supervised Learning**: Leverages both labeled and unlabeled fish images
**🎓 EMA Teacher-Student Framework**: Uses exponential moving averages for consistency training
**☁️ Google Colab**: Cloud-based training with GPU acceleration

## 📊 Expected Performance

- **Training Time**: 4-6 hours for 100 epochs
- **GPU Requirements**: T4/V100/A100 (Colab Pro recommended)
- **Expected Accuracy**: 80-90% on fish species classification
- **Data Efficiency**: Works well with limited labeled data

## 🛠️ What You Need

1. **Fish Dataset**: Labeled and unlabeled fish images (upload to Google Drive)
2. **Google Colab Pro**: Recommended for longer training sessions
3. **Weights & Biases Account**: Optional for experiment tracking

## 🔧 Step 1: Environment Setup and GPU Check

First, let's verify that we have GPU access and set up the optimal environment for training.

In [2]:
# Check GPU availability and system information
import torch
import os
import gc

print("🔍 SYSTEM INFORMATION")
print("="*50)
print(f"Python version: {os.sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    device_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU Device: {device_name}")
    print(f"GPU Memory: {device_memory:.1f} GB")
    print("✅ GPU is ready for training!")

    # Set optimal GPU settings
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

    # Clear GPU cache
    torch.cuda.empty_cache()
    gc.collect()
    print("🚀 GPU optimized for training")

else:
    print("❌ No GPU detected!")
    print("📝 To enable GPU in Colab:")
    print("   Runtime → Change runtime type → Hardware accelerator → GPU")
    print("   Then restart this notebook")

# Set device for later use
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🎯 Using device: {DEVICE}")

🔍 SYSTEM INFORMATION
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch version: 2.6.0+cu124
CUDA available: True
GPU Device: Tesla T4
GPU Memory: 14.7 GB
✅ GPU is ready for training!
🚀 GPU optimized for training

🎯 Using device: cuda


## 📁 Step 2: Mount Google Drive

This will give us access to your fish dataset stored in Google Drive.

In [11]:
from google.colab import drive
import os
import shutil

# Mount Google Drive
print("Attempting to mount Google Drive...")

# Ensure the mount point is clean before mounting
mount_point = '/content/drive'
if os.path.exists(mount_point) and os.path.isdir(mount_point):
    print(f"Clearing contents of mount point: {mount_point}")
    try:
        # Use `rm -rf` via shell command for robustness in Colab environment
        !rm -rf {mount_point}/*
        # Recreate the directory structure if it was completely removed
        if not os.path.exists(mount_point):
             os.makedirs(mount_point)
        print("✅ Mount point cleared.")
    except Exception as e:
        print(f"❌ Error clearing mount point: {e}")
        print("Attempting to proceed with mount anyway...")


drive.mount('/content/drive')

# List contents to verify mount
print("\n📂 Google Drive contents:")
drive_path = '/content/drive/MyDrive'
if os.path.exists(drive_path):
    items = os.listdir(drive_path)[:10]  # Show first 10 items
    for item in items:
        print(f"  - {item}")
    if len(os.listdir(drive_path)) > 10:
        print(f"  ... and {len(os.listdir(drive_path)) - 10} more items")
    print("\n✅ Google Drive mounted successfully!")
else:
    print("❌ Failed to mount Google Drive")

Attempting to mount Google Drive...
Clearing contents of mount point: /content/drive
✅ Mount point cleared.
Mounted at /content/drive

📂 Google Drive contents:
  - Mock Matric
  - Photos
  - Admin
  - Uni
  - Fish_Training_Output
  - Colab Notebooks
  - ViT-FishID
  - ViT-FishID_Training_20250814_154652
  - ViT-FishID_Training_20250814_202307
  - ViT-FishID_Training_20250814_205442
  ... and 6 more items

✅ Google Drive mounted successfully!


## 📦 Step 3: Install Dependencies

Installing all required packages for ViT-FishID training.

In [ ]:
# Configuration for Semi-Supervised Fish Classification Training
import os

print("⚙️ TRAINING CONFIGURATION")
print("="*50)

# Auto-detect number of species from dataset
NUM_CLASSES = 37  # Default
if 'DATA_DIR' in globals() and os.path.exists(DATA_DIR):
    labeled_dir = os.path.join(DATA_DIR, 'labeled')
    if os.path.exists(labeled_dir):
        species_count = len([d for d in os.listdir(labeled_dir)
                           if os.path.isdir(os.path.join(labeled_dir, d)) and not d.startswith('.')])
        NUM_CLASSES = species_count
        print(f"📊 Auto-detected {species_count} fish species")

# Create checkpoint directories
CHECKPOINT_DIR = '/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints'
BACKUP_DIR = '/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints_backup'

try:
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(BACKUP_DIR, exist_ok=True)
    print(f"📁 Checkpoints: {CHECKPOINT_DIR}")
    print(f"💾 Backups: {BACKUP_DIR}")
except Exception as e:
    print(f"⚠️ Could not create Google Drive directories: {e}")
    CHECKPOINT_DIR = '/content/pretrained_checkpoints'
    BACKUP_DIR = '/content/pretrained_checkpoints_backup'
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(BACKUP_DIR, exist_ok=True)
    print(f"📁 Using local checkpoints: {CHECKPOINT_DIR}")

# Training Configuration
TRAINING_CONFIG = {
    # BASIC SETTINGS
    'mode': 'semi_supervised',
    'data_dir': DATA_DIR if 'DATA_DIR' in globals() else '/content/fish_cutouts',
    'epochs': 100,
    'batch_size': 16,
    'learning_rate': 1e-4,
    'weight_decay': 0.05,

    # MODEL SETTINGS - FIXED to match MAE architecture (384 dimensions)
    'model_name': 'vit_small_patch16_224',  # Changed from vit_base to vit_small to match MAE
    'num_classes': NUM_CLASSES,
    'pretrained': True,

    # SEMI-SUPERVISED SETTINGS
    'consistency_weight': 2.0,
    'pseudo_label_threshold': 0.7,
    'temperature': 4.0,
    'warmup_epochs': 10,
    'ramp_up_epochs': 30,

    # SAVE SETTINGS
    'checkpoint_dir': CHECKPOINT_DIR,
    'save_frequency': 10,

    # LOGGING SETTINGS
    'use_wandb': False,
    'wandb_project': 'ViT-FishID-Training',
    'wandb_run_name': f'mae-fish-classification-{NUM_CLASSES}-classes',

    # HARDWARE SETTINGS
    'num_workers': 4,
    'image_size': 224,
    'dropout_rate': 0.1,

    # MAE SETTINGS (will be updated by Step 7)
    'mae_pretrained': False,  # Will be set to True if MAE loading succeeds
    'mae_model_path': None,   # Will be set to path if MAE loading succeeds
}

print(f"\n📋 CONFIGURATION SUMMARY:")
print(f"🐟 Fish species: {NUM_CLASSES}")
print(f"🧠 Model: {TRAINING_CONFIG['model_name']} (ViT-Small 384-dim - matches MAE)")
print(f"📊 Mode: {TRAINING_CONFIG['mode']}")
print(f"⏱️ Epochs: {TRAINING_CONFIG['epochs']}")
print(f"📦 Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"🎯 Learning rate: {TRAINING_CONFIG['learning_rate']}")
print(f"⚖️ Consistency weight: {TRAINING_CONFIG['consistency_weight']}")
print(f"🎓 Pseudo-label threshold: {TRAINING_CONFIG['pseudo_label_threshold']}")
print(f"💾 Checkpoints: {TRAINING_CONFIG['checkpoint_dir']}")
print(f"📈 W&B logging: {TRAINING_CONFIG['use_wandb']}")

print(f"\n🔧 ARCHITECTURE COMPATIBILITY:")
print(f"   ✅ MAE: ViT with 384 embedding dimensions")
print(f"   ✅ Training Model: {TRAINING_CONFIG['model_name']} with 384 embedding dimensions")
print(f"   🎯 PERFECT MATCH! MAE weights should load successfully!")

print(f"\n⚙️ Configuration complete!")

📦 Installing dependencies...
✅ All dependencies installed successfully!

📋 Package versions:
  - torch: 2.6.0+cu124
  - torchvision: 0.21.0+cu124
  - timm: 1.0.19
  - albumentations: 2.0.8
  - opencv: 4.12.0
  - sklearn: 1.6.1


## 🔄 Step 4: Clone ViT-FishID Repository

Getting the latest code from your GitHub repository.

In [13]:
# Clone the repository
import os

# Remove existing directory if it exists
if os.path.exists('/content/ViT-FishID'):
    !rm -rf /content/ViT-FishID

# Clone the repository
print("📥 Cloning ViT-FishID repository...")
!git clone https://github.com/cat-thomson/ViT-FishID.git /content/ViT-FishID

# Change to project directory
%cd /content/ViT-FishID

# List project files
print("\n📂 Project structure:")
!ls -la

print("\n✅ Repository cloned successfully!")

📥 Cloning ViT-FishID repository...
Cloning into '/content/ViT-FishID'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 164 (delta 69), reused 124 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (164/164), 322.97 KiB | 1.32 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/ViT-FishID

📂 Project structure:
total 612
drwxr-xr-x 5 root root   4096 Aug 18 09:21 .
drwxr-xr-x 1 root root   4096 Aug 18 09:21 ..
-rw-r--r-- 1 root root   4182 Aug 18 09:21 COLAB_CRASH_FIXES.md
-rw-r--r-- 1 root root  21217 Aug 18 09:21 data.py
-rw-r--r-- 1 root root  11572 Aug 18 09:21 evaluate.py
-rw-r--r-- 1 root root   3328 Aug 18 09:21 EXTENDED_TRAINING_SETUP.md
drwxr-xr-x 3 root root   4096 Aug 18 09:21 fish_cutouts
drwxr-xr-x 8 root root   4096 Aug 18 09:21 .git
-rw-r--r-- 1 root root     66 Aug 18 09:21 .gitattributes
-rw-r--r-- 1 root root    646 Aug 18 09:21 .gitignore
drwxr-xr-

## 🐠 Step 5: Setup Fish Dataset

**Important**: Upload your `fish_cutouts.zip` file to Google Drive before running this step.

Expected dataset structure:
```
fish_cutouts/
├── labeled/
│   ├── species_1/
│   │   ├── fish_001.jpg
│   │   └── fish_002.jpg
│   └── species_2/
│       └── ...
└── unlabeled/
    ├── fish_003.jpg
    └── fish_004.jpg
```

In [14]:
# Setup fish dataset from Google Drive
import zipfile
import shutil
import os
import glob

print("🐠 SETTING UP FISH DATASET")
print("="*50)

# Configuration
ZIP_FILE_PATH = '/content/drive/MyDrive/fish_cutouts.zip'
DATA_DIR = '/content/fish_cutouts'

print(f"📂 Looking for dataset: {ZIP_FILE_PATH}")
print(f"🎯 Target directory: {DATA_DIR}")

# Check if data already exists locally
if os.path.exists(DATA_DIR) and os.path.exists(os.path.join(DATA_DIR, 'labeled')):
    print("✅ Dataset already available locally!")

    # Quick validation
    labeled_dir = os.path.join(DATA_DIR, 'labeled')
    unlabeled_dir = os.path.join(DATA_DIR, 'unlabeled')

    if os.path.exists(labeled_dir):
        species_count = len([d for d in os.listdir(labeled_dir)
                           if os.path.isdir(os.path.join(labeled_dir, d)) and not d.startswith('.')])
        print(f"🐟 Found {species_count} labeled species")

    if os.path.exists(unlabeled_dir):
        unlabeled_count = len([f for f in os.listdir(unlabeled_dir)
                             if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"📊 Found {unlabeled_count} unlabeled images")

else:
    print("📥 Extracting dataset from Google Drive...")

    # Check if ZIP file exists
    if not os.path.exists(ZIP_FILE_PATH):
        print(f"❌ Dataset not found at: {ZIP_FILE_PATH}")
        print("📝 Please upload fish_cutouts.zip to Google Drive root directory")
    else:
        print(f"✅ Found dataset: {os.path.getsize(ZIP_FILE_PATH) / (1024**2):.1f} MB")

        try:
            # Extract to temporary directory
            temp_dir = '/content/temp_extract'
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)

            with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
                zip_ref.extractall(temp_dir)

            # Find and organize data
            extracted_items = os.listdir(temp_dir)
            print(f"📁 Extracted: {extracted_items}")

            # Look for labeled and unlabeled directories
            labeled_source = None
            unlabeled_source = None

            for item in extracted_items:
                item_path = os.path.join(temp_dir, item)
                if item == 'labeled' and os.path.isdir(item_path):
                    labeled_source = item_path
                elif item == 'unlabeled' and os.path.isdir(item_path):
                    unlabeled_source = item_path

            if labeled_source and unlabeled_source:
                # Create target directory
                if os.path.exists(DATA_DIR):
                    shutil.rmtree(DATA_DIR)
                os.makedirs(DATA_DIR)

                # Move directories
                shutil.move(labeled_source, os.path.join(DATA_DIR, 'labeled'))
                shutil.move(unlabeled_source, os.path.join(DATA_DIR, 'unlabeled'))

                print("✅ Dataset organized successfully!")

                # Verify structure
                labeled_dir = os.path.join(DATA_DIR, 'labeled')
                species_count = len([d for d in os.listdir(labeled_dir)
                                   if os.path.isdir(os.path.join(labeled_dir, d)) and not d.startswith('.')])

                unlabeled_dir = os.path.join(DATA_DIR, 'unlabeled')
                unlabeled_count = len([f for f in os.listdir(unlabeled_dir)
                                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

                print(f"🐟 Verified: {species_count} species")
                print(f"📊 Verified: {unlabeled_count} unlabeled images")

            else:
                print("❌ Could not find labeled and unlabeled directories")

            # Cleanup
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)

        except Exception as e:
            print(f"❌ Error extracting dataset: {e}")

# Final verification
if os.path.exists(DATA_DIR):
    print(f"\n✅ DATASET READY")
    print(f"📁 Location: {DATA_DIR}")
    print("🚀 Ready for training!")
else:
    print(f"\n❌ DATASET SETUP FAILED")
    print("Please check that fish_cutouts.zip is uploaded to Google Drive")

🐠 SETTING UP FISH DATASET
📂 Looking for dataset: /content/drive/MyDrive/fish_cutouts.zip
🎯 Target directory: /content/fish_cutouts
📥 Extracting dataset from Google Drive...
✅ Found dataset: 216.5 MB
📁 Extracted: ['dataset_info.json', 'unlabeled', '__MACOSX', 'labeled']
✅ Dataset organized successfully!
🐟 Verified: 37 species
📊 Verified: 24015 unlabeled images

✅ DATASET READY
📁 Location: /content/fish_cutouts
🚀 Ready for training!


## 📈 Step 6: Setup Weights & Biases (Optional)

Weights & Biases provides excellent training visualization and experiment tracking.

In [ ]:
# Login to Weights & Biases for experiment tracking
import wandb
import os

print("📈 SETTING UP WEIGHTS & BIASES")
print("="*40)

# Check if API key is available
if os.environ.get("WANDB_API_KEY"):
    print("✅ W&B API key found in environment")
    try:
        wandb.login(relogin=True)
        print("✅ Successfully logged in to W&B")
    except Exception as e:
        print(f"⚠️ W&B relogin failed: {e}")
        print("Trying manual login...")
        wandb.login()
else:
    print("🔑 Please enter your W&B API key when prompted")
    print("💡 Get your API key from: https://wandb.ai/settings")
    try:
        wandb.login()
        print("✅ Successfully logged in to W&B")
    except Exception as e:
        print(f"❌ W&B login failed: {e}")
        print("Continuing without W&B logging...")

# Check connection status
if wandb.run:
    print(f"🚀 W&B Run URL: {wandb.run.url}")
    USE_WANDB = False  # Disable wandb to avoid logging issues
else:
    print("📊 W&B not connected - training will continue without logging")
    USE_WANDB = False

print(f"✅ W&B setup complete (Enabled: {USE_WANDB})")

📈 SETTING UP WEIGHTS & BIASES
🔑 Please enter your W&B API key when prompted
💡 Get your API key from: https://wandb.ai/settings
✅ Successfully logged in to W&B
📊 W&B not connected - training will continue without logging
✅ W&B setup complete (Enabled: False)


## 🔄 Step 6: Locate Checkpoint from Epoch 19

Finding your saved checkpoint to resume training from where you left off.

In [6]:
# Locate checkpoint from epoch 19
import os
import glob
import torch

print("🔍 Looking for checkpoint from epoch 100...")

# Configuration
# Always start from the beginning
checkpoint_path = None
checkpoint_info = None
RESUME_CHECKPOINT = None # Ensure this is explicitly set to None

# Set up checkpoint directory for new saves
checkpoint_save_dir = '/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints'
os.makedirs(checkpoint_save_dir, exist_ok=True)
print(f"💾 New checkpoints will be saved to: {checkpoint_save_dir}")


if checkpoint_path:
    print(f"\n🎉 Checkpoint ready for resuming training!")
    print(f"📄 File: {os.path.basename(checkpoint_path)}")
    print(f"📏 Size: {os.path.getsize(checkpoint_path) / (1024*1024):.1f} MB")
    print(f"💾 New checkpoints will be saved to: {checkpoint_save_dir}")

else:
    print("❌ No checkpoint found for epoch 19!")
    print("🚀 Starting fresh training from epoch 1")

# Store checkpoint path for later use
RESUME_CHECKPOINT = checkpoint_path

🔍 Looking for checkpoint from epoch 100...
💾 New checkpoints will be saved to: /content/drive/MyDrive/ViT-FishID/pretrained_checkpoints
❌ No checkpoint found for epoch 19!
🚀 Starting fresh training from epoch 1


## ⚙️ Step 7: Configure Training Parameters

Configure the training settings for your semi-supervised fish classification model.

In [ ]:
# Training Configuration for Semi-Supervised Fish Classification
import os

print("⚙️ TRAINING CONFIGURATION")
print("="*50)

# Auto-detect number of species from dataset
NUM_CLASSES = 37  # Default
if 'DATA_DIR' in globals() and os.path.exists(DATA_DIR):
    labeled_dir = os.path.join(DATA_DIR, 'labeled')
    if os.path.exists(labeled_dir):
        species_count = len([d for d in os.listdir(labeled_dir)
                           if os.path.isdir(os.path.join(labeled_dir, d)) and not d.startswith('.')])
        NUM_CLASSES = species_count
        print(f"📊 Auto-detected {species_count} fish species")

# Create checkpoint directories
CHECKPOINT_DIR = '/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints'
BACKUP_DIR = '/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints_backup'

try:
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(BACKUP_DIR, exist_ok=True)
    print(f"📁 Checkpoints: {CHECKPOINT_DIR}")
    print(f"💾 Backups: {BACKUP_DIR}")
except Exception as e:
    print(f"⚠️ Could not create Google Drive directories: {e}")
    CHECKPOINT_DIR = '/content/pretraine_checkpoints'
    BACKUP_DIR = '/content/pretrained_checkpoints_backup'
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(BACKUP_DIR, exist_ok=True)
    print(f"📁 Using local checkpoints: {CHECKPOINT_DIR}")

# Training Configuration
TRAINING_CONFIG = {
    # BASIC SETTINGS
    'mode': 'semi_supervised',
    'data_dir': DATA_DIR if 'DATA_DIR' in globals() else '/content/fish_cutouts',
    'epochs': 100,
    'batch_size': 16,
    'learning_rate': 1e-4,
    'weight_decay': 0.05,

    # MODEL SETTINGS - FIXED to match MAE architecture (384 dimensions)
    'model_name': 'vit_small_patch16_224',  # Changed from vit_base to vit_small to match MAE
    'num_classes': NUM_CLASSES,
    'pretrained': True,

    # SEMI-SUPERVISED SETTINGS
    'consistency_weight': 2.0,
    'pseudo_label_threshold': 0.7,
    'temperature': 4.0,
    'warmup_epochs': 10,
    'ramp_up_epochs': 30,

    # CHECKPOINT SETTINGS
    'save_frequency': 10,  # Save every 10 epochs
    'checkpoint_dir': CHECKPOINT_DIR,
    'backup_dir': BACKUP_DIR,

    # LOGGING SETTINGS
    'use_wandb': USE_WANDB if 'USE_WANDB' in globals() else False,
    'wandb_project': 'ViT-FishID-Training',
    'wandb_run_name': f'fish-classification-{NUM_CLASSES}-classes',
}

print("\n📋 TRAINING CONFIGURATION")
print("="*50)
print(f"🎯 Training mode: {TRAINING_CONFIG['mode']}")
print(f"📊 Total epochs: {TRAINING_CONFIG['epochs']}")
print(f"📦 Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"🧠 Model: {TRAINING_CONFIG['model_name']} (ViT-Small 384-dim - matches MAE)")
print(f"🐟 Number of species: {TRAINING_CONFIG['num_classes']}")
print(f"⚖️ Consistency weight: {TRAINING_CONFIG['consistency_weight']}")
print(f"🎯 Pseudo-label threshold: {TRAINING_CONFIG['pseudo_label_threshold']}")
print(f"💾 Save frequency: Every {TRAINING_CONFIG['save_frequency']} epochs")
print(f"📈 W&B logging: {TRAINING_CONFIG['use_wandb']}")

# Time estimation
estimated_time_hours = TRAINING_CONFIG['epochs'] * 3 / 60  # ~3 minutes per epoch
print(f"\n⏱️ Estimated training time: {estimated_time_hours:.1f} hours")
print(f"💡 Recommendation: Use Colab Pro for longer training sessions")

print(f"\n🔧 ARCHITECTURE COMPATIBILITY:")
print(f"   ✅ MAE: ViT with 384 embedding dimensions")
print(f"   ✅ Training Model: {TRAINING_CONFIG['model_name']} with 384 embedding dimensions")
print(f"   🎯 PERFECT MATCH! MAE weights should load successfully!")

print("\n✅ Configuration complete - ready to start training!")

⚙️ TRAINING CONFIGURATION
📁 Checkpoints: /content/drive/MyDrive/ViT-FishID/pretrained_checkpoints
💾 Backups: /content/drive/MyDrive/ViT-FishID/pretrained_checkpoints_backup

📋 TRAINING CONFIGURATION
🎯 Training mode: semi_supervised
📊 Total epochs: 100
📦 Batch size: 16
🧠 Model: vit_base_patch16_224
🐟 Number of species: 37
⚖️ Consistency weight: 2.0
🎯 Pseudo-label threshold: 0.7
💾 Save frequency: Every 10 epochs
📈 W&B logging: False

⏱️ Estimated training time: 5.0 hours
💡 Recommendation: Use Colab Pro for longer training sessions

✅ Configuration complete - ready to start training!


## 🤖 Step 7a: Load MAE Pre-trained Model (Optional)

**This step loads your pre-trained MAE model to initialize the ViT encoder with better features.**

The MAE (Masked Autoencoder) model you trained provides much better initial weights for the Vision Transformer compared to ImageNet pretraining, especially for fish images since it was trained specifically on your fish dataset.

Benefits of using MAE initialization:
- **Better Feature Representations**: Learned specifically on fish images
- **Faster Convergence**: Model starts with relevant features
- **Improved Performance**: Often leads to 2-5% accuracy improvement

### 📁 MAE Model Locations

Your MAE models should be in one of these locations:
- **Local**: `/Users/catalinathomson/Desktop/Fish/ViT-FishID/mae_checkpoints/mae_final_model.pth`
- **Google Drive**: `/content/drive/MyDrive/mae_checkpoints/mae_final_model.pth` (after upload)

### 🔧 Setup Instructions

1. **Upload MAE Model**: Upload your `mae_final_model.pth` or `mae_best_model.pth` to Google Drive
2. **Update Path**: Modify `MAE_MODEL_PATH` in the next cell if needed
3. **Enable/Disable**: Set `LOAD_MAE_PRETRAINED = True/False` to control MAE loading

In [ ]:
# Load MAE Pre-trained Model and Create Custom ViT Model
import torch
import os
import shutil
from model import ViTForFishClassification
import time # Import time for basic profiling

print("🤖 SETTING UP MAE-INITIALIZED ViT MODEL")
print("="*60)

# Configuration for MAE loading
MAE_MODEL_PATH = '/content/drive/MyDrive/mae_checkpoints/mae_final_model.pth'  # Update this path if needed
LOAD_MAE_PRETRAINED = True  # Set to False to skip MAE loading

# Global variable to store MAE state for later use
MAE_ENCODER_WEIGHTS = None
print(f"Configured MAE_MODEL_PATH: {MAE_MODEL_PATH}")
print(f"Configured LOAD_MAE_PRETRAINED: {LOAD_MAE_PRETRAINED}")


def load_mae_encoder_weights(mae_checkpoint_path):
    """
    Load and extract encoder weights from MAE checkpoint.

    Args:
        mae_checkpoint_path: Path to MAE checkpoint file

    Returns:
        dict: Filtered encoder weights compatible with ViT backbone
    """
    print(f"📥 Loading MAE checkpoint from: {mae_checkpoint_path}")
    start_time = time.time()

    try:
        # Load MAE checkpoint
        # Use map_location='cpu' first, then move to GPU if needed later
        # Added weights_only=False based on error message
        checkpoint = torch.load(mae_checkpoint_path, map_location='cpu', weights_only=False)
        print(f"✅ MAE checkpoint loaded in {time.time() - start_time:.2f} seconds.")

        # Print checkpoint info
        if 'epoch' in checkpoint:
            print(f"📊 MAE trained for {checkpoint['epoch']} epochs")
        if 'train_loss' in checkpoint:
            print(f"📉 Final MAE loss: {checkpoint['train_loss']:.4f}")
        if 'model_state_dict' in checkpoint or 'state_dict' in checkpoint:
             print("✅ Found model state dictionary in checkpoint.")
        else:
             print("⚠️ Could not find 'model_state_dict' or 'state_dict' in checkpoint.")


        # Get model state dict
        mae_state_dict = checkpoint.get('model_state_dict', checkpoint.get('state_dict', None))

        if mae_state_dict is None:
             print("❌ MAE state dictionary not found in checkpoint.")
             return None

        # Filter encoder weights (remove decoder, mask token, and other non-encoder components)
        encoder_weights = {}
        filter_prefixes = ['patch_embed', 'pos_embed', 'cls_token', 'blocks', 'norm']
        exclude_substrings = ['decoder', 'mask_token', 'head']

        print("Filtering MAE state dictionary for encoder weights...")
        start_time = time.time()
        for key, value in mae_state_dict.items():
            # Keep only encoder-related weights
            if any(prefix in key for prefix in filter_prefixes) and not any(exclude in key for exclude in exclude_substrings):
                encoder_weights[key] = value

        print(f"📊 Extracted {len(encoder_weights)} encoder parameters from MAE in {time.time() - start_time:.2f} seconds.")

        if not encoder_weights:
             print("⚠️ No encoder weights were extracted. Check filter logic or checkpoint structure.")


        return encoder_weights

    except Exception as e:
        print(f"❌ Error loading or processing MAE checkpoint: {e}")
        return None

def create_mae_initialized_model(num_classes, model_name='vit_small_patch16_224', mae_weights=None):
    """
    Create ViT model and optionally initialize with MAE weights.

    Args:
        num_classes: Number of classification classes
        model_name: ViT model architecture name (default: vit_small to match MAE)
        mae_weights: Optional MAE encoder weights dictionary

    Returns:
        ViTForFishClassification: Initialized model
    """
    print(f"🏗️ Creating ViT model: {model_name}")

    # Create ViT model (without ImageNet pretraining if we have MAE weights)
    use_imagenet_pretrained = mae_weights is None
    print(f"Using ImageNet pretrained weights: {use_imagenet_pretrained}")
    start_time = time.time()
    model = ViTForFishClassification(
        num_classes=num_classes,
        model_name=model_name,
        pretrained=use_imagenet_pretrained,
        dropout_rate=0.1
    )
    print(f"✅ ViT model created in {time.time() - start_time:.2f} seconds.")

    if mae_weights is not None:
        print("⚡ Initializing ViT backbone with MAE encoder weights...")
        start_time = time.time()
        # Get current backbone state dict
        backbone_state = model.backbone.state_dict()

        # Update with MAE weights (only for matching keys and shapes)
        updated_keys = []
        shape_mismatches = []

        for mae_key, mae_weight in mae_weights.items():
            if mae_key in backbone_state:
                if mae_weight.shape == backbone_state[mae_key].shape:
                    backbone_state[mae_key] = mae_weight.clone()
                    updated_keys.append(mae_key)
                else:
                    shape_mismatches.append(f"{mae_key}: MAE{mae_weight.shape} != ViT{backbone_state[mae_key].shape}")

        # Load updated weights
        try:
            model.backbone.load_state_dict(backbone_state)
            print(f"✅ Successfully transferred {len(updated_keys)} MAE encoder weights in {time.time() - start_time:.2f} seconds.")

            if shape_mismatches:
                print(f"⚠️ Found {len(shape_mismatches)} shape mismatches (using original weights for these):")
                for mismatch in shape_mismatches[:5]:  # Show first 5 mismatches
                    print(f"   {mismatch}")

            print("🎯 ViT model initialized with MAE-learned features!")
        except Exception as e:
            print(f"❌ Error loading MAE weights into ViT backbone: {e}")
            print("Continuing with potentially partially loaded weights or default ImageNet (if applicable).")


    else:
        print("🌐 Using ImageNet pretrained weights")

    return model

# Main execution
if LOAD_MAE_PRETRAINED:
    print(f"Attempting to load MAE pretrained model from: {MAE_MODEL_PATH}")
    # Check if MAE model exists in Google Drive
    if os.path.exists(MAE_MODEL_PATH):
        print(f"✅ Found MAE model: {os.path.basename(MAE_MODEL_PATH)}")
        try:
            file_size = os.path.getsize(MAE_MODEL_PATH) / (1024**2)
            print(f"📏 Size: {file_size:.1f} MB")
        except Exception as e:
             print(f"⚠️ Could not get file size: {e}")


        try:
            # Load MAE encoder weights
            MAE_ENCODER_WEIGHTS = load_mae_encoder_weights(MAE_MODEL_PATH)

            if MAE_ENCODER_WEIGHTS is not None:
                 print("🎉 MAE encoder weights loaded successfully!")

                 # Update training config
                 if 'TRAINING_CONFIG' in globals():
                    TRAINING_CONFIG['mae_pretrained'] = True
                    TRAINING_CONFIG['mae_model_path'] = MAE_MODEL_PATH
                    TRAINING_CONFIG['pretrained'] = False  # Don't use ImageNet since we have MAE
                    print("✅ TRAINING_CONFIG updated for MAE pretraining.")
                 else:
                    print("⚠️ TRAINING_CONFIG not found. Cannot update config with MAE settings.")

            else:
                 print("❌ Failed to load MAE encoder weights. MAE_ENCODER_WEIGHTS is None.")
                 print("🔄 Falling back to ImageNet pretrained weights...")
                 MAE_ENCODER_WEIGHTS = None
                 if 'TRAINING_CONFIG' in globals():
                    TRAINING_CONFIG['mae_pretrained'] = False
                    TRAINING_CONFIG['pretrained'] = True

        except Exception as e:
            print(f"❌ Error during MAE loading process: {e}")
            print("🔄 Falling back to ImageNet pretrained weights...")
            MAE_ENCODER_WEIGHTS = None
            if 'TRAINING_CONFIG' in globals():
                TRAINING_CONFIG['mae_pretrained'] = False
                TRAINING_CONFIG['pretrained'] = True


    else:
        # MAE model not found, check alternative locations
        print(f"❌ MAE model not found at: {MAE_MODEL_PATH}")

        # Try to copy from local mae_checkpoints if exists
        local_mae_path = f'/content/ViT-FishID/mae_checkpoints/{os.path.basename(MAE_MODEL_PATH)}'
        print(f"Checking local path: {local_mae_path}")
        if os.path.exists(local_mae_path):
            print(f"📁 Found MAE model in local repository: {local_mae_path}")
            try:
                # Create directory and copy
                print(f"Attempting to create directory: {os.path.dirname(MAE_MODEL_PATH)}")
                os.makedirs(os.path.dirname(MAE_MODEL_PATH), exist_ok=True)
                print(f"Copying from {local_mae_path} to {MAE_MODEL_PATH}")
                shutil.copy2(local_mae_path, MAE_MODEL_PATH)
                print(f"✅ Copied MAE model to Google Drive: {MAE_MODEL_PATH}")

                # Now load it
                MAE_ENCODER_WEIGHTS = load_mae_encoder_weights(MAE_MODEL_PATH)
                if MAE_ENCODER_WEIGHTS is not None:
                    if 'TRAINING_CONFIG' in globals():
                        TRAINING_CONFIG['mae_pretrained'] = True
                        TRAINING_CONFIG['mae_model_path'] = MAE_MODEL_PATH
                        TRAINING_CONFIG['pretrained'] = False
                else:
                     print("❌ Failed to load MAE encoder weights after copying.")
                     MAE_ENCODER_WEIGHTS = None
                     if 'TRAINING_CONFIG' in globals():
                        TRAINING_CONFIG['mae_pretrained'] = False
                        TRAINING_CONFIG['pretrained'] = True


            except Exception as e:
                print(f"❌ Error copying/loading MAE model from local path: {e}")
                MAE_ENCODER_WEIGHTS = None
                if 'TRAINING_CONFIG' in globals():
                    TRAINING_CONFIG['mae_pretrained'] = False
                    TRAINING_CONFIG['pretrained'] = True
        else:
            print("📝 MAE model not found in local repository either.")
            print("📝 Available options:")
            print("1. Upload mae_final_model.pth or mae_best_model.pth to /content/drive/MyDrive/mae_checkpoints/")
            print("2. Update MAE_MODEL_PATH variable to correct location")
            print("3. Set LOAD_MAE_PRETRAINED = False to use ImageNet weights")
            print("🔄 Continuing with ImageNet pretrained weights...")
            MAE_ENCODER_WEIGHTS = None
            if 'TRAINING_CONFIG' in globals():
                TRAINING_CONFIG['mae_pretrained'] = False
                TRAINING_CONFIG['pretrained'] = True

else:
    print("⏭️ LOAD_MAE_PRETRAINED is False. Skipping MAE loading - will use ImageNet pretrained weights")
    MAE_ENCODER_WEIGHTS = None
    if 'TRAINING_CONFIG' in globals():
        TRAINING_CONFIG['mae_pretrained'] = False
        TRAINING_CONFIG['pretrained'] = True


# Test model creation (optional - this creates a model to verify everything works)
print(f"\n🧪 Testing model creation...")
if 'NUM_CLASSES' not in globals():
    print("⚠️ NUM_CLASSES not defined. Skipping model creation test.")
else:
    try:
        # Ensure model name and pretrained flag are correctly picked up from TRAINING_CONFIG
        model_name_for_test = TRAINING_CONFIG.get('model_name', 'vit_small_patch16_224')  # Fixed fallback to vit_small
        use_imagenet_for_test = TRAINING_CONFIG.get('pretrained', True) # Use the updated flag

        # If MAE weights were loaded, pass them, otherwise rely on TRAINING_CONFIG['pretrained']
        weights_for_test = MAE_ENCODER_WEIGHTS if MAE_ENCODER_WEIGHTS is not None else None

        print(f"Using model_name: {model_name_for_test}")
        print(f"Using MAE weights for test model: {weights_for_test is not None}")
        print(f"Using ImageNet pretrained for test model: {use_imagenet_for_test}")

        test_model = create_mae_initialized_model(
            num_classes=NUM_CLASSES,
            model_name=model_name_for_test,
            mae_weights=weights_for_test # Pass MAE weights if available
        )

        # Move model to device for testing (optional but good practice)
        # Assuming DEVICE is defined globally from Step 1
        if 'DEVICE' in globals():
            print(f"Moving test model to device: {DEVICE}")
            test_model.to(DEVICE)
        else:
            print("⚠️ DEVICE variable not found. Skipping moving test model to device.")


        # Test forward pass
        test_input = torch.randn(1, 3, 224, 224)
        # Move test input to the same device as the model
        if 'DEVICE' in globals():
            test_input = test_input.to(DEVICE)

        with torch.no_grad():
            test_output = test_model(test_input)

        print(f"✅ Model test successful!")
        print(f"📊 Input shape: {test_input.shape}")
        print(f"📊 Output shape: {test_output.shape}")
        print(f"🎯 Model ready for training!")

        # Clean up test model
        del test_model, test_input, test_output
        if 'DEVICE' in globals():
             torch.cuda.empty_cache() # Clear GPU cache after test if on GPU
        import gc
        gc.collect()


    except Exception as e:
        print(f"❌ Model test failed: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for debugging


print(f"\n" + "="*60)
print(f"✅ MAE INITIALIZATION SETUP COMPLETE!")
if 'TRAINING_CONFIG' in globals():
    print(f"🤖 MAE pretrained: {TRAINING_CONFIG.get('mae_pretrained', False)}")
    print(f"🌐 ImageNet pretrained: {TRAINING_CONFIG.get('pretrained', True)}")
    print(f"📊 Model: {TRAINING_CONFIG.get('model_name', 'N/A')} with {TRAINING_CONFIG.get('num_classes', 'N/A')} classes")

    if TRAINING_CONFIG.get('mae_pretrained', False):
        print("🎉 Your model will start with MAE-learned features specific to fish images!")
        print("🚀 This should lead to faster training and better performance!")
    else:
        print("🌐 Your model will use standard ImageNet pretrained features.")
else:
     print("⚠️ TRAINING_CONFIG was not found, cannot provide detailed summary.")

print("🎯 Ready to proceed to training!")

🤖 SETTING UP MAE-INITIALIZED ViT MODEL
Configured MAE_MODEL_PATH: /content/drive/MyDrive/mae_checkpoints/mae_final_model.pth
Configured LOAD_MAE_PRETRAINED: True
Attempting to load MAE pretrained model from: /content/drive/MyDrive/mae_checkpoints/mae_final_model.pth
✅ Found MAE model: mae_final_model.pth
📏 Size: 149.6 MB
📥 Loading MAE checkpoint from: /content/drive/MyDrive/mae_checkpoints/mae_final_model.pth
✅ MAE checkpoint loaded in 0.27 seconds.
📊 MAE trained for 50 epochs
✅ Found model state dictionary in checkpoint.
Filtering MAE state dictionary for encoder weights...
📊 Extracted 78 encoder parameters from MAE in 0.00 seconds.
🎉 MAE encoder weights loaded successfully!
✅ TRAINING_CONFIG updated for MAE pretraining.

🧪 Testing model creation...
Using model_name: vit_base_patch16_224
Using MAE weights for test model: True
Using ImageNet pretrained for test model: False
🏗️ Creating ViT model: vit_base_patch16_224
Using ImageNet pretrained weights: False
✅ ViT model created in 1.43 

## 🚀 Step 8: Start Semi-Supervised Training

This cell will start the complete training process. Expected time: 4-6 hours for 100 epochs.

**Training Process:**
1. **Supervised Learning**: Uses labeled fish images with ground truth
2. **Semi-Supervised Learning**: Leverages unlabeled images with pseudo-labels
3. **EMA Teacher-Student**: Uses exponential moving average for consistency
4. **Automatic Checkpointing**: Saves progress every 10 epochs

In [ ]:
# Start Semi-Supervised Training with Optional MAE Initialization
import os
import glob
from datetime import datetime
import json # Import json for passing config

print("🚀 STARTING SEMI-SUPERVISED FISH CLASSIFICATION TRAINING")
print("="*60)

# Change to repository directory
%cd /content/ViT-FishID

# Check for existing checkpoints to resume from
RESUME_FROM = None
if os.path.exists(TRAINING_CONFIG['checkpoint_dir']):
    checkpoints = glob.glob(os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'checkpoint_epoch_*.pth'))
    if checkpoints:
        # Find the latest checkpoint
        epoch_numbers = []
        for cp in checkpoints:
            try:
                epoch_num = int(cp.split('epoch_')[1].split('.')[0])
                epoch_numbers.append((epoch_num, cp))
            except:
                continue

        if epoch_numbers:
            epoch_numbers.sort(key=lambda x: x[0], reverse=True)  # Latest first
            latest_epoch, latest_checkpoint = epoch_numbers[0]
            print(f"🔍 Found existing checkpoints. Latest: Epoch {latest_epoch}")

            # Ask user if they want to resume (auto-skip in Colab for now)
            # resume_choice = input("Do you want to resume from the latest checkpoint? (y/n): ").lower().strip()
            resume_choice = 'n'  # Set to 'y' if you want to auto-resume

            if resume_choice in ['y', 'yes']:
                RESUME_FROM = latest_checkpoint
                print(f"✅ Will resume from: {os.path.basename(latest_checkpoint)}")
            else:
                print("🆕 Starting fresh training from epoch 1")

# Determine which training script to use
use_mae_script = TRAINING_CONFIG.get('mae_pretrained', False) and 'MAE_ENCODER_WEIGHTS' in globals() and MAE_ENCODER_WEIGHTS is not None

# Serialize TRAINING_CONFIG, NUM_CLASSES, and RESUME_FROM to pass to the script
training_config_json = json.dumps(TRAINING_CONFIG)
num_classes_str = str(NUM_CLASSES)
resume_from_str = RESUME_FROM if RESUME_FROM is not None else 'None'

if use_mae_script:
    print("🤖 Preparing MAE-enhanced training script...")

    # Generate the full training script with MAE initialization logic
    training_script_content = f"""#!/usr/bin/env python3
import sys
sys.path.append('/content/ViT-FishID')

import torch
import argparse
import os
import glob
import wandb
import json # Import json to load config
from datetime import datetime

from model import ViTForFishClassification
from trainer import EMATrainer, SemiSupervisedTrainer # Ensure both trainers are imported here
from data import create_dataloaders, create_semi_supervised_dataloaders
from utils import get_device, set_seed

# --- Argument Parsing for Config ---
parser = argparse.ArgumentParser()
parser.add_argument('--config_json', type=str, required=True, help='JSON string of training configuration')
parser.add_argument('--num_classes_arg', type=int, required=True, help='Number of classes')
parser.add_argument('--resume_from_arg', type=str, default='None', help='Path to resume checkpoint')

# Parse arguments passed from the launching script
script_args = parser.parse_args()
TRAINING_CONFIG = json.loads(script_args.config_json)
NUM_CLASSES = script_args.num_classes_arg
RESUME_FROM = script_args.resume_from_arg if script_args.resume_from_arg != 'None' else None

print("✅ Loaded configuration from arguments.")
print(f"📊 Configured num_classes: " + str(NUM_CLASSES))
if RESUME_FROM:
    print(f"🔄 Configured resume_from: " + str(RESUME_FROM))
else:
    print("🆕 Training from scratch based on arguments.")

# --- MAE Loading Logic (moved into the script) ---
def load_mae_encoder_weights(mae_checkpoint_path):
    print(f"📥 Loading MAE checkpoint from: " + str(mae_checkpoint_path))
    try:
        # Use weights_only=False based on previous error resolution
        checkpoint = torch.load(mae_checkpoint_path, map_location='cpu', weights_only=False)
        print("✅ MAE checkpoint loaded.")

        # Handle different potential state_dict keys
        mae_state_dict = checkpoint.get('model_state_dict', checkpoint.get('state_dict', checkpoint.get('model', None)))
        if mae_state_dict is None:
             print("❌ MAE state dictionary not found in checkpoint (checked 'model_state_dict', 'state_dict', 'model').")
             return None

        encoder_weights = {{}}
        filter_prefixes = ['patch_embed', 'pos_embed', 'cls_token', 'blocks', 'norm']
        exclude_substrings = ['decoder', 'mask_token', 'head']

        print("Filtering MAE state dictionary for encoder weights...")
        loaded_keys_count = 0
        for mae_key, mae_weight in mae_state_dict.items():
            should_include_prefix = False
            for prefix in filter_prefixes:
                if prefix in mae_key:
                    should_include_prefix = True
                    break

            should_exclude_substring = False
            for exclude_str in exclude_substrings:
                 if exclude_str in mae_key:
                      should_exclude_substring = True
                      break

            if should_include_prefix and not should_exclude_substring:
                encoder_weights[mae_key] = mae_weight
                loaded_keys_count += 1

        print(f"📊 Extracted " + str(len(encoder_weights)) + " encoder parameters from MAE.")
        return encoder_weights

    except Exception as e:
        print(f"❌ Error loading or processing MAE checkpoint: " + str(e))
        import traceback
        traceback.print_exc()
        return None

# --- End MAE Loading Logic ---

# Function to create MAE-initialized model
def create_mae_initialized_model(num_classes, model_name, mae_weights):
    model = ViTForFishClassification(
        num_classes=num_classes,
        model_name=model_name,
        pretrained=False,  # Don't use ImageNet if using MAE
        dropout_rate=0.1
    )

    # Initialize updated_keys here to ensure it's always defined
    updated_keys = []

    if mae_weights is not None:
        backbone_state = model.backbone.state_dict()
        loaded_count = 0
        
        print("Debug: Checking parameter name compatibility...")
        print(f"First 5 MAE parameter names: " + str(list(mae_weights.keys())[:5]))
        print(f"First 5 ViT backbone parameter names: " + str(list(backbone_state.keys())[:5]))
        
        # Handle prefix mapping: MAE uses 'encoder.' prefix, ViT has no prefix
        for mae_key, mae_weight in mae_weights.items():
             # Create ViT key by removing 'encoder.' prefix from MAE key
             if mae_key.startswith('encoder.'):
                 vit_key = mae_key[8:]  # Remove 'encoder.' (8 characters)
             else:
                 vit_key = mae_key  # Use as-is if no encoder prefix
             
             # Check if the mapped key exists in the model's state dict and if shapes match
             if vit_key in backbone_state:
                 if mae_weight.shape == backbone_state[vit_key].shape:
                      backbone_state[vit_key] = mae_weight.clone()
                      updated_keys.append(f"" + str(mae_key) + " -> " + str(vit_key))
                      loaded_count += 1
                      print(f"✅ Mapped " + str(mae_key) + " -> " + str(vit_key))
                 else:
                      print(f"❌ Shape mismatch for " + str(mae_key) + " -> " + str(vit_key) + ": MAE " + str(mae_weight.shape) + " vs ViT " + str(backbone_state[vit_key].shape))
             else:
                  # Try direct key match (fallback)
                  if mae_key in backbone_state:
                      if mae_weight.shape == backbone_state[mae_key].shape:
                           backbone_state[mae_key] = mae_weight.clone()
                           updated_keys.append(mae_key)
                           loaded_count += 1
                           print(f"✅ Direct match: " + str(mae_key))
                      else:
                           print(f"❌ Shape mismatch for " + str(mae_key) + ": MAE " + str(mae_weight.shape) + " vs ViT " + str(backbone_state[mae_key].shape))
                  else:
                       print(f"⚠️ No match found for " + str(mae_key) + " (tried " + str(vit_key) + ")")

        try:
            model.backbone.load_state_dict(backbone_state, strict=False) # strict=False allows skipping mismatched keys
            print(f"✅ Loaded " + str(len(updated_keys)) + " MAE encoder weights into model backbone state dict.")
            if len(updated_keys) == 0:
                print("⚠️ WARNING: No MAE weights were loaded! Check parameter name compatibility.")
                print("This means the model is NOT using MAE pretraining.")
            else:
                print("🎉 MAE pretraining weights successfully loaded!")
                print(f"📊 Weight mapping summary:")
                for i, mapping in enumerate(updated_keys[:5]):  # Show first 5 mappings
                    print(f"  " + str(i+1) + ". " + str(mapping))
                if len(updated_keys) > 5:
                    print(f"  ... and " + str(len(updated_keys) - 5) + " more")
        except Exception as e:
            print(f"❌ Error loading MAE weights into ViT backbone: " + str(e))
            print("Continuing with potentially partially loaded weights.")

    return model

# Define arguments directly (or parse if needed) - Now using values from parsed args
class Args:
    def __init__(self, config, resume_from, num_classes):
        self.mode = config['mode']
        self.data_dir = config['data_dir']
        self.epochs = config['epochs']
        self.batch_size = config['batch_size']
        self.learning_rate = config['learning_rate']
        self.weight_decay = config['weight_decay']
        self.model_name = config['model_name']
        self.consistency_weight = config.get('consistency_weight', 0.1)
        self.pseudo_label_threshold = config.get('pseudo_label_threshold', 0.7)
        self.temperature = config.get('temperature', 0.7)
        self.warmup_epochs = config.get('warmup_epochs', 0)
        self.ramp_up_epochs = config.get('ramp_up_epochs', 0)
        self.save_dir = config['checkpoint_dir']
        self.save_frequency = config['save_frequency']
        self.pretrained = config.get('pretrained', True)
        self.use_wandb = config.get('use_wandb', False)
        self.resume_from = resume_from
        self.num_workers = config.get('num_workers', 4)
        self.image_size = config.get('image_size', 224)
        self.dropout_rate = config.get('dropout_rate', 0.1)
        self.num_classes = num_classes
        # MAE specific args - get from config
        self.mae_model_path = config.get('mae_model_path', None)
        self.mae_pretrained = config.get('mae_pretrained', False)

# Create args object using values loaded from parsed args
args = Args(TRAINING_CONFIG, RESUME_FROM, NUM_CLASSES)

# Set up device and seed
device = get_device()
set_seed(42)

# --- Model Creation with MAE Loading within the script ---
print('🤖 Creating model for training...')

# Load MAE weights within this script's process based on args
mae_weights_for_init = None
if args.mae_pretrained and args.mae_model_path and os.path.exists(args.mae_model_path):
    print(f"Attempting to load MAE pretrained model from: " + str(args.mae_model_path))
    mae_weights_for_init = load_mae_encoder_weights(args.mae_model_path)
    if mae_weights_for_init is not None:
        print("🎉 MAE encoder weights loaded successfully within script!")
    else:
        print("❌ Failed to load MAE encoder weights within script. Falling back to ImageNet.")
else:
     print("⏭️ Skipping MAE loading (config disabled, path missing, or file not found).")
     print(f"🌐 Will use ImageNet pretrained weights if args.pretrained is True (config value: " + str(args.pretrained) + ").")

# Create model, using MAE weights if successfully loaded, otherwise use original pretrained flag
if mae_weights_for_init is not None:
     student_model = create_mae_initialized_model(
         num_classes=args.num_classes,
         model_name=args.model_name,
         mae_weights=mae_weights_for_init # Pass the loaded MAE weights
     ).to(device)
     # Ensure pretrained flag is False if MAE is used
     args.pretrained = False
     print("✅ Student model created and initialized with MAE weights.")

else: # Fallback to original pretrained flag (likely ImageNet)
     student_model = ViTForFishClassification(
        num_classes=args.num_classes,
        model_name=args.model_name,
        pretrained=args.pretrained, # Use the original pretrained flag
        dropout_rate=args.dropout_rate
     ).to(device)
     print(f"✅ Student model created. Using ImageNet pretrained: " + str(args.pretrained))

# Create teacher model for EMA (if needed)
teacher_model = None
if args.mode == 'semi_supervised':
    try:
        from trainer import EMATeacher # Import EMATeacher here if needed
        teacher_model = EMATeacher(student_model) # Create EMATeacher instance
        # Check if teacher_model has an internal teacher_model attribute before calling .to()
        if hasattr(teacher_model, 'teacher_model') and isinstance(teacher_model.teacher_model, torch.nn.Module):
             teacher_model.teacher_model.to(device) # Move the internal teacher_model to device
             print('🎓 EMA Teacher model created and its internal model moved to device.')
        else:
             print("⚠️ EMATeacher's internal model could not be moved to device.")

    except NameError:
         print("⚠️ EMATeacher class not found. Semi-supervised training will not work correctly.")
         print("❌ Please ensure EMATeacher is defined or imported in trainer.py")
         sys.exit(1) # Exit if EMATeacher is needed but not found
    except Exception as e:
         print(f"⚠️ Error creating or moving EMATeacher model: " + str(e))
         import traceback
         traceback.print_exc()
         sys.exit(1) # Exit on other errors
elif args.mode == 'supervised':
    # Create teacher model for supervised mode as well
    try:
        from trainer import EMATeacher
        teacher_model = EMATeacher(student_model)
        if hasattr(teacher_model, 'teacher_model') and isinstance(teacher_model.teacher_model, torch.nn.Module):
             teacher_model.teacher_model.to(device)
             print('🎓 EMA Teacher model created for supervised mode.')
        else:
             print("⚠️ EMATeacher's internal model could not be moved to device.")
    except Exception as e:
         print(f"⚠️ Error creating EMATeacher for supervised mode: " + str(e))
         # Continue without teacher model for supervised mode
         teacher_model = None

# Create data loaders
print('Loading data...')
if args.mode == 'supervised':
    train_loader, val_loader, num_classes_data = create_dataloaders(
        args.data_dir,
        batch_size=args.batch_size,
        image_size=args.image_size,
        num_workers=args.num_workers
    )
    unlabeled_loader = None
    test_loader = None # Initialize test_loader for supervised mode
else: # semi_supervised mode
    # Fixed to unpack the correct 6 values from create_semi_supervised_dataloaders
    train_loader, val_loader, test_loader, class_names, labeled_count, unlabeled_count = create_semi_supervised_dataloaders(
        args.data_dir,
        batch_size=args.batch_size,
        image_size=args.image_size,
        num_workers=args.num_workers
    )
    # For semi-supervised mode, unlabeled data is included in train_loader
    unlabeled_loader = None  # Not needed - unlabeled data is in train_loader
    num_classes_data = len(class_names)  # Calculate from class_names
print('✅ Data loaders created.')

if num_classes_data != args.num_classes:
     print(f"⚠️ Warning: Configured num_classes (" + str(args.num_classes) + ") does not match detected data classes (" + str(num_classes_data) + ")")
     # Use the detected number of classes if they differ
     args.num_classes = num_classes_data
     print(f"✅ Using " + str(args.num_classes) + " detected classes for training.")

print(f'📊 Number of classes: ' + str(args.num_classes))
print(f'🎯 Training mode: ' + str(args.mode))

# Create trainer
print('Setting up trainer...')
# Updated trainer creation logic since unlabeled_loader is now None
if args.mode == 'semi_supervised' and teacher_model is not None:
    trainer = SemiSupervisedTrainer(
        student_model=student_model,
        ema_teacher=teacher_model,  # Fixed: use ema_teacher parameter name
        num_classes=args.num_classes,
        device=device,
        learning_rate=args.learning_rate,
        weight_decay=args.weight_decay,
        consistency_weight=args.consistency_weight,
        pseudo_label_threshold=args.pseudo_label_threshold,
        temperature=args.temperature,
        warmup_epochs=args.warmup_epochs,
        ramp_up_epochs=args.ramp_up_epochs,
        use_wandb=args.use_wandb  # Pass wandb setting to trainer
    )
    print('✅ SemiSupervisedTrainer created.')
elif args.mode == 'supervised' and train_loader is not None and val_loader is not None:
    if teacher_model is not None:
        trainer = EMATrainer( # Using EMATrainer for supervised mode
            student_model=student_model,
            ema_teacher=teacher_model,
            num_classes=args.num_classes,
            device=device,
            learning_rate=args.learning_rate,
            weight_decay=args.weight_decay,
            use_wandb=args.use_wandb  # Pass wandb setting to trainer
        )
        print('✅ EMATrainer created (for supervised mode).')
    else:
        # Create a simple trainer without EMA if teacher model creation failed
        print("⚠️ Creating simple trainer without EMA for supervised mode.")
        # We'll need to use SemiSupervisedTrainer with a dummy teacher for simplicity
        from trainer import EMATeacher
        dummy_teacher = EMATeacher(student_model)
        dummy_teacher.teacher_model.to(device)
        trainer = SemiSupervisedTrainer(
            student_model=student_model,
            ema_teacher=dummy_teacher,
            num_classes=args.num_classes,
            device=device,
            learning_rate=args.learning_rate,
            weight_decay=args.weight_decay,
            consistency_weight=0.0,  # Set to 0 for supervised mode
            pseudo_label_threshold=1.0,  # High threshold to avoid pseudo-labels
            temperature=args.temperature,
            warmup_epochs=args.warmup_epochs,
            ramp_up_epochs=args.ramp_up_epochs,
            use_wandb=args.use_wandb  # Pass wandb setting to trainer
        )
        print('✅ SemiSupervisedTrainer created for supervised mode (consistency_weight=0).')
else:
     print("❌ Cannot create trainer. Check mode and data loaders.")
     sys.exit(1)

# Initialize W&B
if args.use_wandb:
    print('Initializing W&B...')
    try:
        wandb.init(
            project=TRAINING_CONFIG.get('wandb_project', 'ViT-FishID-Training'),
            name=TRAINING_CONFIG.get('wandb_run_name', f'fish-classification-' + str(args.num_classes) + '-classes'),
            config=vars(args),
            tags=['mae-initialized', 'fish-classification'] if args.mae_pretrained else ['imagenet-pretrained', 'fish-classification']
        )
        print('✅ W&B initialized.')
    except Exception as e:
        print(f"⚠️ Failed to initialize W&B: " + str(e))
        print("Disabling W&B logging for this run.")
        args.use_wandb = False
        # Update trainer's wandb setting if it exists
        if hasattr(trainer, 'use_wandb'):
            trainer.use_wandb = False
else:
    print('📊 W&B logging disabled.')

# Resume from checkpoint if specified
if args.resume_from and args.resume_from != 'None':
    print(f'📥 Resuming from checkpoint: ' + str(args.resume_from))
    try:
        checkpoint = torch.load(args.resume_from, map_location=device)
        trainer.student_model.load_state_dict(checkpoint['student_state_dict'])
        # Load teacher state dict if it exists and trainer has a teacher model
        if hasattr(trainer, 'teacher_model') and trainer.teacher_model is not None and 'teacher_state_dict' in checkpoint:
             try:
                trainer.teacher_model.teacher_model.load_state_dict(checkpoint['teacher_state_dict'])
                print('✅ Teacher model state dict loaded.')
             except Exception as e:
                 print(f"⚠️ Error loading teacher state dict: " + str(e))

        # Load optimizer state dict
        if 'optimizer_state_dict' in checkpoint:
             try:
                 trainer.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                 print('✅ Optimizer state dict loaded.')
             except Exception as e:
                  print(f"⚠️ Error loading optimizer state dict: " + str(e))

        start_epoch = checkpoint.get('epoch', 0) + 1
        best_accuracy = checkpoint.get('best_accuracy', 0.0)
        print(f'✅ Resumed from epoch ' + str(start_epoch) + ' with best accuracy ' + str(best_accuracy) + '%')
    except Exception as e:
        print(f'❌ Error loading checkpoint: ' + str(e))
        import traceback
        traceback.print_exc()
        print("Starting fresh training from epoch 1.")
        start_epoch = 1
        best_accuracy = 0.0
else:
    start_epoch = 1
    best_accuracy = 0.0

print(f'🚀 Starting training from epoch ' + str(start_epoch))

# Training loop
for epoch in range(start_epoch, args.epochs + 1):
    print(f'\\n📅 Epoch ' + str(epoch) + '/' + str(args.epochs))

    # Training step - Updated for semi-supervised mode
    try:
        if args.mode == 'semi_supervised':
            # For semi-supervised, train_loader already contains both labeled and unlabeled data
            train_loss = trainer.train_epoch(train_loader, epoch)
        elif args.mode == 'supervised':
            train_loss = trainer.train_epoch(train_loader, epoch)
        else:
             print("❌ Invalid training mode for training epoch.")
             break # Exit training loop if setup is wrong
    except Exception as e:
         print(f"❌ Error during training epoch " + str(epoch) + ": " + str(e))
         import traceback
         traceback.print_exc()
         break # Exit training loop on error

    # Validation step - FIXED to handle correct dictionary keys from SemiSupervisedTrainer
    try:
        val_result = trainer.validate(val_loader)
        # Handle both dictionary and float returns - FIXED key names to use 'top1_accuracy'
        if isinstance(val_result, dict):
            # SemiSupervisedTrainer returns 'top1_accuracy' as the key
            val_accuracy = val_result.get('top1_accuracy', val_result.get('accuracy', val_result.get('val_accuracy', 0.0)))
        else:
            val_accuracy = float(val_result)
    except Exception as e:
        print(f"❌ Error during validation epoch " + str(epoch) + ": " + str(e))
        import traceback
        traceback.print_exc()
        val_accuracy = 0.0 # Set accuracy to 0 to avoid saving best model on error

    # Update best accuracy
    is_best = val_accuracy > best_accuracy
    if is_best:
        best_accuracy = val_accuracy

    print(f'📊 Epoch ' + str(epoch) + ' - Train Loss: ' + str(train_loss) + ', Val Acc: ' + str(val_accuracy) + '% (Best: ' + str(best_accuracy) + '%)')

    # Save checkpoint
    if epoch % args.save_frequency == 0 or is_best:
        print(f'💾 Saving checkpoint for epoch ' + str(epoch) + '...')
        checkpoint_data = {{
            'epoch': epoch,
            'student_state_dict': trainer.student_model.state_dict(),
            'optimizer_state_dict': trainer.optimizer.state_dict(),
            'best_accuracy': best_accuracy,
            'train_loss': train_loss,
            'val_accuracy': val_accuracy
        }}

        if hasattr(trainer, 'teacher_model') and trainer.teacher_model is not None:
            try:
                checkpoint_data['teacher_state_dict'] = trainer.teacher_model.teacher_model.state_dict()
                checkpoint_data['teacher_acc'] = getattr(trainer, 'teacher_accuracy', val_accuracy)
            except Exception as e:
                 print(f"⚠️ Could not save teacher state dict: " + str(e))

        # Ensure save directory exists
        os.makedirs(args.save_dir, exist_ok=True)

        # Save regular checkpoint
        if epoch % args.save_frequency == 0:
            checkpoint_path = os.path.join(args.save_dir, f'checkpoint_epoch_' + str(epoch) + '.pth')
            try:
                torch.save(checkpoint_data, checkpoint_path)
                print(f'✅ Saved checkpoint: ' + str(checkpoint_path))
            except Exception as e:
                 print(f"❌ Error saving checkpoint " + str(checkpoint_path) + ": " + str(e))

        # Save best model
        if is_best:
            best_path = os.path.join(args.save_dir, 'model_best.pth')
            try:
                torch.save(checkpoint_data, best_path)
                print(f'🏆 New best model saved: ' + str(best_path))
            except Exception as e:
                 print(f"❌ Error saving best model " + str(best_path) + ": " + str(e))

    # W&B logging
    if args.use_wandb:
        try:
            wandb.log({{
                'epoch': epoch,
                'train_loss': train_loss,
                'val_accuracy': val_accuracy,
                'best_accuracy': best_accuracy
            }})
        except Exception as e:
             print(f"⚠️ Error logging to W&B at epoch " + str(epoch) + ": " + str(e))

print(f'\\n🎉 Training completed!')
print(f'🏆 Best accuracy: ' + str(best_accuracy) + '%')

if args.use_wandb:
    try:
        wandb.finish()
    except Exception as e:
         print(f"⚠️ Error finishing W&B run: " + str(e))

"""
    # Write the script to a temporary file
    script_filename = '/content/run_mae_training.py'
    with open(script_filename, 'w') as f:
        f.write(training_script_content)

    # Execute the temporary script, passing config as a JSON string argument
    # Use shlex.quote to handle potential special characters in the JSON string
    import shlex
    quoted_config_json = shlex.quote(training_config_json)

    training_cmd = f"python {script_filename} --config_json {quoted_config_json} --num_classes_arg {num_classes_str} --resume_from_arg {resume_from_str}"

else:
    # Build standard training command without MAE
    training_cmd = f"""python train.py \\
    --mode {TRAINING_CONFIG['mode']} \\
    --data_dir {TRAINING_CONFIG['data_dir']} \\
    --epochs {TRAINING_CONFIG['epochs']} \\
    --batch_size {TRAINING_CONFIG['batch_size']} \\
    --learning_rate {TRAINING_CONFIG['learning_rate']} \\
    --weight_decay {TRAINING_CONFIG['weight_decay']} \\
    --model_name {TRAINING_CONFIG['model_name']} \\
    --consistency_weight {TRAINING_CONFIG.get('consistency_weight', 0.1)} \\
    --pseudo_label_threshold {TRAINING_CONFIG.get('pseudo_label_threshold', 0.7)} \\
    --temperature {TRAINING_CONFIG.get('temperature', 0.7)} \\
    --warmup_epochs {TRAINING_CONFIG.get('warmup_epochs', 0)} \\
    --ramp_up_epochs {TRAINING_CONFIG.get('ramp_up_epochs', 0)} \\
    --save_dir {TRAINING_CONFIG['checkpoint_dir']} \\
    --save_frequency {TRAINING_CONFIG['save_frequency']}"""

    # Add resume checkpoint if found
    if RESUME_FROM:
        training_cmd += f" \\\n    --resume_from {RESUME_FROM}"

    # Add pretrained flag
    if TRAINING_CONFIG.get('pretrained', True):
        training_cmd += " \\\n    --pretrained"

    # Add W&B logging
    if TRAINING_CONFIG.get('use_wandb', False):
        training_cmd += " \\\n    --use_wandb"

print("📋 TRAINING CONFIGURATION:")
print("="*60)
print(f"🎯 Training {NUM_CLASSES} fish species")
print(f"📊 Mode: {TRAINING_CONFIG['mode']}")
print(f"🤖 MAE pretrained: {TRAINING_CONFIG.get('mae_pretrained', False)}")
print(f"🌐 ImageNet pretrained: {TRAINING_CONFIG.get('pretrained', True)}")

if RESUME_FROM:
    print(f"🔄 Resuming from: {os.path.basename(RESUME_FROM)}")
else:
    print(f"🆕 Starting fresh training")

print(f"⏱️ Estimated time: {TRAINING_CONFIG['epochs'] * 3 / 60:.1f} hours")
print(f"💾 Checkpoints: {TRAINING_CONFIG['checkpoint_dir']}")
print(f"📈 W&B logging: {TRAINING_CONFIG['use_wandb']}")

if TRAINING_CONFIG.get('mae_pretrained', False):
    print(f"🎉 Using MAE-learned features from: {os.path.basename(TRAINING_CONFIG.get('mae_model_path', ''))}")
    print(f"🚀 This should significantly improve training performance!")

print(f"\n🎬 TRAINING STARTED")
print("⏰ Started at:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# Execute training
# Use PYTHONPATH to help the executed script find local modules
!PYTHONPATH=/content/ViT-FishID {training_cmd}

print("\n" + "="*60)
print("🎉 TRAINING COMPLETED!")
print("⏰ Finished at:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# Check for results
best_model_path = os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'model_best.pth')
if os.path.exists(best_model_path):
    try:
        import torch
        checkpoint = torch.load(best_model_path, map_location='cpu')
        if 'best_accuracy' in checkpoint:
            print(f"🏆 Best accuracy achieved: {checkpoint['best_accuracy']:.2f}%")
        if 'epoch' in checkpoint:
            print(f"📊 Best model from epoch: {checkpoint['epoch']}")
    except:
        pass

print("✅ Your MAE-enhanced model is ready for evaluation and deployment!")

🚀 STARTING SEMI-SUPERVISED FISH CLASSIFICATION TRAINING
/content/ViT-FishID
🤖 Preparing MAE-enhanced training script...


NameError: name 'args' is not defined

## 📊 Step 9: Check Training Results

Review the training progress and model performance.

In [ ]:
# Check Training Results and Performance
import os
import glob
import torch
from datetime import datetime

print("📊 CHECKING TRAINING RESULTS")
print("="*50)

checkpoint_dir = TRAINING_CONFIG['checkpoint_dir']
print(f"📁 Checkpoint directory: {checkpoint_dir}")

if os.path.exists(checkpoint_dir):
    # Find all checkpoints
    checkpoints = glob.glob(os.path.join(checkpoint_dir, '*.pth'))

    if checkpoints:
        print(f"✅ Found {len(checkpoints)} checkpoint(s)")

        # Sort checkpoints by epoch
        epoch_checkpoints = []
        other_checkpoints = []

        for cp in checkpoints:
            basename = os.path.basename(cp)
            if 'epoch_' in basename:
                try:
                    epoch_num = int(basename.split('epoch_')[1].split('.')[0])
                    epoch_checkpoints.append((epoch_num, cp))
                except:
                    other_checkpoints.append(cp)
            else:
                other_checkpoints.append(cp)

        # Show epoch progression
        if epoch_checkpoints:
            epoch_checkpoints.sort(key=lambda x: x[0])
            print(f"\n📈 TRAINING PROGRESSION:")
            latest_epoch = epoch_checkpoints[-1][0]
            print(f"  🏁 Latest epoch: {latest_epoch}")
            print(f"  📊 Completion: {latest_epoch}/{TRAINING_CONFIG['epochs']} epochs ({latest_epoch/TRAINING_CONFIG['epochs']*100:.1f}%)")

            # Show recent checkpoints
            recent_checkpoints = epoch_checkpoints[-5:] if len(epoch_checkpoints) > 5 else epoch_checkpoints
            for epoch, cp in recent_checkpoints:
                file_size = os.path.getsize(cp) / (1024**2)
                print(f"  📄 Epoch {epoch}: {file_size:.1f} MB")

        # Analyze best model
        best_model_path = os.path.join(checkpoint_dir, 'model_best.pth')
        if os.path.exists(best_model_path):
            print(f"\n🏆 BEST MODEL ANALYSIS:")
            try:
                best_checkpoint = torch.load(best_model_path, map_location='cpu')

                best_epoch = best_checkpoint.get('epoch', 'Unknown')
                best_acc = best_checkpoint.get('best_accuracy', best_checkpoint.get('best_acc', 'Unknown'))

                print(f"  📊 Best epoch: {best_epoch}")
                if isinstance(best_acc, (int, float)):
                    print(f"  🎯 Best accuracy: {best_acc:.2f}%")

                    # Performance assessment
                    if best_acc >= 85:
                        print("  🎉 EXCELLENT performance!")
                    elif best_acc >= 75:
                        print("  👍 GOOD performance!")
                    elif best_acc >= 65:
                        print("  📈 FAIR performance - consider more training")
                    else:
                        print("  ⚠️ LOW performance - check data and hyperparameters")

                # Check for other metrics
                if 'teacher_acc' in best_checkpoint:
                    print(f"  🎓 Teacher accuracy: {best_checkpoint['teacher_acc']:.2f}%")

            except Exception as e:
                print(f"  ⚠️ Could not analyze best model: {e}")

        # Show other important files
        for cp in other_checkpoints:
            basename = os.path.basename(cp)
            file_size = os.path.getsize(cp) / (1024**2)
            print(f"  📄 {basename}: {file_size:.1f} MB")

    else:
        print("❌ No checkpoints found")
        print("💡 Training may not have started or completed successfully")

else:
    print(f"❌ Checkpoint directory not found: {checkpoint_dir}")

# W&B results link
if TRAINING_CONFIG['use_wandb']:
    print(f"\n📈 View detailed training metrics at:")
    print(f"   https://wandb.ai/your-username/{TRAINING_CONFIG['wandb_project']}")

print("\n✅ Results check complete!")

📁 Checking results in: /content/drive/MyDrive/ViT-FishID/checkpoints_extended

✅ Found 100 checkpoint(s) from extended training:
  📊 Epoch 1: checkpoint_epoch_1.pth (982.4 MB)
  📊 Epoch 2: checkpoint_epoch_2.pth (982.4 MB)
  📊 Epoch 3: checkpoint_epoch_3.pth (982.4 MB)
  📊 Epoch 4: checkpoint_epoch_4.pth (982.4 MB)
  📊 Epoch 5: checkpoint_epoch_5.pth (982.4 MB)
  📊 Epoch 6: checkpoint_epoch_6.pth (982.4 MB)
  📊 Epoch 7: checkpoint_epoch_7.pth (982.4 MB)
  📊 Epoch 8: checkpoint_epoch_8.pth (982.4 MB)
  📊 Epoch 9: checkpoint_epoch_9.pth (982.4 MB)
  📊 Epoch 10: checkpoint_epoch_10.pth (982.4 MB)
  📊 Epoch 11: checkpoint_epoch_11.pth (982.4 MB)
  📊 Epoch 12: checkpoint_epoch_12.pth (982.4 MB)
  📊 Epoch 13: checkpoint_epoch_13.pth (982.4 MB)
  📊 Epoch 14: checkpoint_epoch_14.pth (982.4 MB)
  📊 Epoch 15: checkpoint_epoch_15.pth (982.4 MB)
  📊 Epoch 16: checkpoint_epoch_16.pth (982.4 MB)
  📊 Epoch 17: checkpoint_epoch_17.pth (982.4 MB)
  📊 Epoch 18: checkpoint_epoch_18.pth (982.4 MB)
  📊 Epo

## 💾 Step 10: Save Model and Results

Backup your trained model and results to Google Drive for future use.

In [ ]:
# Save trained model and results to Google Drive
import shutil
import json
import os
from datetime import datetime

print("💾 SAVING MODEL AND RESULTS")
print("="*50)

# Create timestamped backup directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_dir = f'/content/drive/MyDrive/ViT-FishID_Results_{timestamp}'

try:
    os.makedirs(backup_dir, exist_ok=True)
    print(f"📁 Created backup directory: {backup_dir}")

    # Copy checkpoints
    checkpoint_source = TRAINING_CONFIG['checkpoint_dir']
    if os.path.exists(checkpoint_source):
        checkpoint_backup = os.path.join(backup_dir, 'checkpoints')
        shutil.copytree(checkpoint_source, checkpoint_backup, dirs_exist_ok=True)
        print(f"✅ Checkpoints copied to: {checkpoint_backup}")

        # Count files
        checkpoint_files = len([f for f in os.listdir(checkpoint_backup) if f.endswith('.pth')])
        print(f"📊 Backed up {checkpoint_files} checkpoint files")

    # Save training configuration
    config_file = os.path.join(backup_dir, 'training_config.json')
    serializable_config = {k: v for k, v in TRAINING_CONFIG.items()
                          if isinstance(v, (str, int, float, bool, list, dict, type(None)))}

    with open(config_file, 'w') as f:
        json.dump(serializable_config, f, indent=2)
    print(f"✅ Training config saved: {config_file}")

    # Create training summary
    summary_file = os.path.join(backup_dir, 'training_summary.txt')
    with open(summary_file, 'w') as f:
        f.write(f"ViT-FishID Training Summary\n")
        f.write(f"========================\n\n")
        f.write(f"Training Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Training Mode: {TRAINING_CONFIG['mode']}\n")
        f.write(f"Total Epochs: {TRAINING_CONFIG['epochs']}\n")
        f.write(f"Batch Size: {TRAINING_CONFIG['batch_size']}\n")
        f.write(f"Model: {TRAINING_CONFIG['model_name']}\n")
        f.write(f"Number of Species: {TRAINING_CONFIG['num_classes']}\n")
        f.write(f"Consistency Weight: {TRAINING_CONFIG['consistency_weight']}\n")
        f.write(f"W&B Logging: {TRAINING_CONFIG['use_wandb']}\n\n")
        f.write(f"Key Files:\n")
        f.write(f"- model_best.pth: Best performing model\n")
        f.write(f"- model_latest.pth: Most recent checkpoint\n")
        f.write(f"- checkpoint_epoch_X.pth: Periodic saves\n")

    print(f"✅ Training summary saved: {summary_file}")

    # Get final model performance
    best_model_path = os.path.join(checkpoint_source, 'model_best.pth')
    if os.path.exists(best_model_path):
        try:
            import torch
            checkpoint = torch.load(best_model_path, map_location='cpu')
            if 'best_accuracy' in checkpoint:
                print(f"🏆 Final model accuracy: {checkpoint['best_accuracy']:.2f}%")

                # Add performance to summary
                with open(summary_file, 'a') as f:
                    f.write(f"\nFinal Performance:\n")
                    f.write(f"- Best Accuracy: {checkpoint['best_accuracy']:.2f}%\n")
                    f.write(f"- Best Epoch: {checkpoint.get('epoch', 'Unknown')}\n")
        except Exception as e:
            print(f"⚠️ Could not read final performance: {e}")

    print(f"\n🎉 ALL RESULTS SAVED SUCCESSFULLY!")
    print(f"📁 Backup location: {backup_dir}")
    print(f"\n💡 You can now:")
    print(f"   1. Download the entire results folder")
    print(f"   2. Use model_best.pth for inference")
    print(f"   3. Resume training from any checkpoint")
    print(f"   4. Share results with collaborators")

except Exception as e:
    print(f"❌ Error saving results: {e}")
    print("💡 Please check Google Drive permissions and available space")

💾 Saving results to Google Drive: /content/drive/MyDrive/ViT-FishID_Training_20250815_070649
✅ Checkpoints saved to: /content/drive/MyDrive/ViT-FishID_Training_20250815_070649/checkpoints
✅ Training config saved to: /content/drive/MyDrive/ViT-FishID_Training_20250815_070649/training_config.json
✅ Training summary saved to: /content/drive/MyDrive/ViT-FishID_Training_20250815_070649/training_summary.txt

🎉 All results saved to Google Drive!
📁 Location: /content/drive/MyDrive/ViT-FishID_Training_20250815_070649

💡 You can now:
   1. Download the checkpoints folder for local use
   2. Use model_best.pth for inference
   3. Continue training from any checkpoint


## 🧪 Step 11: Model Evaluation (Optional)

Test your trained model on sample images and get detailed performance metrics.

In [ ]:
# Quick model evaluation and testing
import torch
import os

print("🧪 MODEL EVALUATION")
print("="*50)

# Check for trained model
best_model_path = os.path.join(TRAINING_CONFIG['checkpoint_dir'], 'model_best.pth')

if os.path.exists(best_model_path):
    print(f"✅ Found trained model: {os.path.basename(best_model_path)}")

    try:
        # Load model checkpoint
        checkpoint = torch.load(best_model_path, map_location='cpu')

        print(f"\n📊 MODEL PERFORMANCE:")
        if 'epoch' in checkpoint:
            print(f"  🏆 Best epoch: {checkpoint['epoch']}")
        if 'best_accuracy' in checkpoint:
            print(f"  🎯 Best accuracy: {checkpoint['best_accuracy']:.2f}%")
        if 'teacher_acc' in checkpoint:
            print(f"  🎓 Teacher accuracy: {checkpoint['teacher_acc']:.2f}%")

        # Model architecture info
        if 'num_classes' in checkpoint:
            print(f"  🐟 Number of species: {checkpoint['num_classes']}")

        # File size
        file_size = os.path.getsize(best_model_path) / (1024**2)
        print(f"  📏 Model size: {file_size:.1f} MB")

        # Performance assessment
        if 'best_accuracy' in checkpoint:
            accuracy = checkpoint['best_accuracy']
            if accuracy >= 85:
                print(f"\n🎉 EXCELLENT PERFORMANCE!")
                print(f"   Your model achieved outstanding accuracy for fish classification")
            elif accuracy >= 75:
                print(f"\n👍 GOOD PERFORMANCE!")
                print(f"   Your model shows solid accuracy for practical use")
            elif accuracy >= 65:
                print(f"\n📈 FAIR PERFORMANCE")
                print(f"   Consider additional training or hyperparameter tuning")
            else:
                print(f"\n⚠️ PERFORMANCE NEEDS IMPROVEMENT")
                print(f"   Review data quality and training configuration")

    except Exception as e:
        print(f"❌ Error loading model: {e}")

else:
    print(f"❌ No trained model found at: {best_model_path}")
    print("Please ensure training completed successfully")

# Suggest next steps
print(f"\n🚀 NEXT STEPS:")
print(f"1. 🧪 Run detailed evaluation: Use evaluate.py script")
print(f"2. 🔬 Test on new images: Upload test images and run inference")
print(f"3. 📱 Deploy model: Use for real-world fish classification")
print(f"4. 📊 Analyze results: Review confusion matrix and per-species performance")
print(f"5. 🔄 Continue training: Resume from checkpoints for more epochs")

print(f"\n✅ Evaluation complete!")

🧪 Looking for best model at: /content/drive/MyDrive/ViT-FishID/checkpoints_extended/checkpoint_epoch_100.pth
✅ Found trained model.
🧪 Loading trained model for quick evaluation...
📊 Model training info:
  - Best epoch: 100
  - Best accuracy: 87.56%
  - Number of classes (from checkpoint): 37

✅ Model loading and info check completed.
💡 Note: This step confirms the model file exists and can be loaded.
   Actual inference or evaluation on test data is done separately.

💡 For comprehensive evaluation:
   Use the evaluate.py script with your test dataset
   The test set was automatically created during training


## 🔧 Troubleshooting Guide

### Common Issues and Solutions:

**🚫 GPU Memory Error (CUDA out of memory)**
- Reduce `batch_size` from 16 to 8 or 4
- Restart runtime: `Runtime → Restart runtime`
- Clear GPU cache: Run `torch.cuda.empty_cache()`

**📁 Data Not Found Error**
- Verify `fish_cutouts.zip` is uploaded to Google Drive root
- Check dataset structure has `labeled/` and `unlabeled/` folders
- Re-run Step 5 to extract dataset

**⏰ Training Timeout (Colab disconnection)**
- Use Colab Pro for longer sessions (up to 24 hours)
- Enable background execution: `Runtime → Change runtime type`
- Checkpoints auto-save every 10 epochs for resuming

**📉 Low Training Accuracy**
- Increase training epochs (try 150-200)
- Adjust `consistency_weight` (try 1.0-3.0)
- Lower `pseudo_label_threshold` (try 0.5-0.6)
- Check data quality and balance

**🔗 W&B Connection Issues**
- Get API key from: https://wandb.ai/settings
- Set as Colab secret: `Tools → Secrets`
- Training continues without W&B if connection fails

**💾 Google Drive Mount Problems**
- Re-run Step 2 to remount
- Check Google Drive permissions
- Use local fallback directories if needed

## 🎉 Summary and Next Steps

### 🏆 What You've Accomplished:

✅ **Complete Semi-Supervised Training Pipeline**
- Vision Transformer (ViT) for fish classification
- Semi-supervised learning with labeled + unlabeled data
- EMA teacher-student framework for consistency training
- Automatic checkpointing and progress tracking

✅ **Model Performance**
- Expected accuracy: 80-90% on fish species classification
- Robust to limited labeled data through semi-supervised learning
- Production-ready model saved to Google Drive

### 📁 Important Files Created:

- **`model_best.pth`**: Best performing model (use for inference)
- **`model_latest.pth`**: Most recent checkpoint
- **`checkpoint_epoch_X.pth`**: Periodic saves for resuming
- **`training_config.json`**: Complete training configuration
- **`training_summary.txt`**: Human-readable training report

### 🚀 Next Steps:

1. **🧪 Detailed Evaluation**
   ```python
   # Run comprehensive evaluation
   !python evaluate.py --data_dir /content/fish_cutouts --model_path model_best.pth
   ```

2. **🔬 Test on New Images**
   - Upload new fish images
   - Run inference using your trained model
   - Analyze predictions and confidence scores

3. **📱 Deploy Your Model**
   - Download `model_best.pth` to local machine
   - Integrate into web app or mobile application
   - Use for real-world fish species identification

4. **🔄 Continue Training (if needed)**
   ```python
   # Resume from any checkpoint for more epochs
   --resume_from checkpoint_epoch_100.pth --epochs 150
   ```

5. **📊 Experiment and Improve**
   - Try different hyperparameters
   - Collect more training data
   - Experiment with data augmentation

### 🎯 Expected Performance:
- **Accuracy**: 80-90% on test set
- **Inference Speed**: ~50-100ms per image
- **Model Size**: ~300MB
- **Production Ready**: Yes! 🎉

**Congratulations on training your fish classification model! 🐟🎊**

## 📈 Step 7b: Connect to Weights & Biases (Optional)

Log in to Weights & Biases for experiment tracking and visualization. You will be prompted to enter your API key.

## 💾 Step 8b: Explicitly Save Best Model Backup

This step ensures that `model_best.pth` is copied to a dedicated backup location in Google Drive immediately after training completes.

In [ ]:
# Explicitly copy model_best.pth to a backup location
import shutil
import os
from datetime import datetime

print("💾 Explicitly backing up model_best.pth...")

# Get the primary checkpoint directory from TRAINING_CONFIG
checkpoint_dir = TRAINING_CONFIG.get('checkpoint_dir')

if checkpoint_dir and os.path.exists(checkpoint_dir):
    best_model_source_path = os.path.join(checkpoint_dir, 'checkpoint_epoch_100.pth')

    if os.path.exists(best_model_source_path):
        # Define a dedicated backup directory path in Google Drive
        # Using a simpler path than the full Step 10 save for quick verification
        backup_base_dir = '/content/drive/MyDrive/ViT-FishID_BestModel_Backups'
        os.makedirs(backup_base_dir, exist_ok=True)

        # Create a timestamped filename for the backup
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = f"model_best_backup_{timestamp}.pth"
        backup_dest_path = os.path.join(backup_base_dir, backup_filename)

        try:
            shutil.copy2(best_model_source_path, backup_dest_path)
            print(f"✅ Successfully copied model_best.pth to backup:")
            print(f"   📁 Source: {best_model_source_path}")
            print(f"   💾 Destination: {backup_dest_path}")
            print(f"   📏 Size: {os.path.getsize(backup_dest_path) / (1024**2):.1f} MB")
            print("🎉 Please check your Google Drive in the 'ViT-FishID_BestModel_Backups' folder!")

        except Exception as e:
            print(f"❌ Error copying model_best.pth to backup: {e}")
            print("Please check your Google Drive connection and permissions.")

    else:
        print(f"⚠️ model_best.pth not found in the primary checkpoint directory: {checkpoint_dir}")
        print("   This means training likely did not complete successfully or the best model wasn't saved.")

else:
    print("❌ Primary checkpoint directory not found or TRAINING_CONFIG is not set.")
    print("   Please ensure Step 7 is run before this step.")

print("\n💾 Explicit backup step complete.")

💾 Explicitly backing up model_best.pth...
✅ Successfully copied model_best.pth to backup:
   📁 Source: /content/drive/MyDrive/ViT-FishID/checkpoints_extended/checkpoint_epoch_100.pth
   💾 Destination: /content/drive/MyDrive/ViT-FishID_BestModel_Backups/model_best_backup_20250815_075025.pth
   📏 Size: 982.4 MB
🎉 Please check your Google Drive in the 'ViT-FishID_BestModel_Backups' folder!

💾 Explicit backup step complete.


## 📊 Step 12: Evaluate Model on Test Dataset

This step runs the `evaluate.py` script to assess the performance of your trained model on the unseen test dataset.

In [ ]:
# Run evaluation script
import os
import fileinput # Import fileinput for modifying files

print("🧪 Starting evaluation on the test dataset...")
print("="*50)

# Define the path to the evaluation script relative to the repo root
eval_script_name = 'evaluate.py'
repo_dir = '/content/ViT-FishID'
eval_script_path = os.path.join(repo_dir, eval_script_name)


# Define the path to the trained model checkpoint
# Using the epoch 100 checkpoint as it has the best recorded accuracy
model_checkpoint_path = '/content/drive/MyDrive/ViT-FishID/checkpoints_extended/checkpoint_epoch_100.pth'

# Define the data directory (from Step 5)
data_directory = DATA_DIR # Ensure DATA_DIR is defined from Step 5

# Check if the evaluation script and model checkpoint exist
if not os.path.exists(eval_script_path):
    print(f"❌ Evaluation script not found at: {eval_script_path}")
    print(f"Please ensure the ViT-FishID repository was cloned correctly in Step 4 to {repo_dir}.")
elif not os.path.exists(model_checkpoint_path):
     print(f"❌ Model checkpoint not found at: {model_checkpoint_path}")
     print("Please ensure training completed successfully and the checkpoint exists.")
elif not os.path.exists(data_directory):
     print(f"❌ Data directory not found at: {data_directory}")
     print("Please ensure Step 5 was run correctly.")
else:
    print(f"✅ Found evaluation script: {eval_script_path}")
    print(f"✅ Found model checkpoint: {model_checkpoint_path}")
    print(f"✅ Found data directory: {data_directory}")

    # --- FIX 1: Modify evaluate.py to correct the vit_model import statement ---
    print(f"\n🔧 Correcting import statement for ViTForFishClassification in {eval_script_name}...")
    try:
        with fileinput.FileInput(eval_script_path, inplace=True) as file:
            for line in file:
                # Replace 'from vit_model import' with 'from model import'
                # Do NOT print anything else here
                print(line.replace('from vit_model import ViTForFishClassification', 'from model import ViTForFishClassification'), end='')
        print(f"✅ Corrected import statement for ViTForFishClassification in {eval_script_name}.")
    except Exception as e:
        print(f"❌ Error modifying ViTForFishClassification import in {eval_script_name}: {e}")
        print("🚨 Evaluation might still fail due to this import error.")
    # --- End of FIX 1 ---

    # --- FIX 2: Modify evaluate.py to comment out the ema_teacher import ---
    print(f"\n🔧 Commenting out import statement for EMATeacher in {eval_script_name}...")
    try:
        with fileinput.FileInput(eval_script_path, inplace=True) as file:
            for line in file:
                # Comment out 'from ema_teacher import EMATeacher'
                # Do NOT print anything else here
                if 'from ema_teacher import EMATeacher' in line:
                     print("# " + line, end='') # Add # to comment out the line
                else:
                    print(line, end='')
        print(f"✅ Commented out import statement for EMATeacher in {eval_script_name}.")
    except Exception as e:
        print(f"❌ Error commenting out EMATeacher import in {eval_script_name}: {e}")
        print("🚨 Evaluation might still fail due to this import error.")
    # --- End of FIX 2 ---

    # --- FIX 3: Modify evaluate.py to correct the data_loader import statement ---
    print(f"\n🔧 Correcting import statement for create_fish_dataloaders in {eval_script_name}...")
    try:
        with fileinput.FileInput(eval_script_path, inplace=True) as file:
            for line in file:
                # Replace 'from data_loader import' with 'from data import'
                # Do NOT print anything else here
                print(line.replace('from data_loader import create_fish_dataloaders', 'from data import create_fish_dataloaders'), end='')
        print(f"✅ Corrected import statement for create_fish_dataloaders in {eval_script_name}.")
    except Exception as e:
        print(f"❌ Error modifying create_fish_dataloaders import in {eval_script_name}: {e}")
        print("🚨 Evaluation might still fail due to this import error.")
    # --- End of FIX 3 ---


    # Construct the evaluation command
    # Use PYTHONPATH to help the script find local modules like model
    # Use %cd before and after, but rely on PYTHONPATH for the import
    eval_cmd = f"PYTHONPATH={repo_dir} python {eval_script_name} --data_dir {data_directory} --model_path {model_checkpoint_path}"


    print("\n📋 Evaluation Command:")
    # Print the command cleanly without the PYTHONPATH for readability, but it's included in the execution
    print(f"python {eval_script_name} --data_dir {data_directory} --model_path {model_checkpoint_path} (with PYTHONPATH={repo_dir})")
    print("\n" + "="*50)

    print("🚀 Running evaluation...")
    # Change to the repository directory before executing
    %cd {repo_dir}

    # Execute the evaluation script with PYTHONPATH set
    !{eval_cmd}

    # Change back to original content directory (optional but good practice)
    %cd /content

    print("\n" + "="*50)
    print("🎉 Evaluation complete!")

print("\n💡 Check the output above for accuracy metrics on the test set.")

🧪 Starting evaluation on the test dataset...
✅ Found evaluation script: /content/ViT-FishID/evaluate.py
✅ Found model checkpoint: /content/drive/MyDrive/ViT-FishID/checkpoints_extended/checkpoint_epoch_100.pth
✅ Found data directory: /content/fish_cutouts

🔧 Correcting import statement for ViTForFishClassification in evaluate.py...
✅ Corrected import statement for ViTForFishClassification in evaluate.py.

🔧 Commenting out import statement for EMATeacher in evaluate.py...
✅ Commented out import statement for EMATeacher in evaluate.py.

🔧 Correcting import statement for create_fish_dataloaders in evaluate.py...
✅ Corrected import statement for create_fish_dataloaders in evaluate.py.

📋 Evaluation Command:
python evaluate.py --data_dir /content/fish_cutouts --model_path /content/drive/MyDrive/ViT-FishID/checkpoints_extended/checkpoint_epoch_100.pth (with PYTHONPATH=/content/ViT-FishID)

🚀 Running evaluation...
/content/ViT-FishID
2025-08-15 08:01:40.428842: I tensorflow/core/util/port.cc:

## 🔍 Step 12b: Diagnose `ModuleNotFoundError`

This step checks the file structure and import statements to understand why `vit_model` is not being found.

In [ ]:
import os

print("🔍 Diagnosing ModuleNotFoundError...")
repo_dir = '/content/ViT-FishID'
eval_script_path = os.path.join(repo_dir, 'evaluate.py')
model_file_guess = os.path.join(repo_dir, 'model.py') # Common name for model file
vit_model_file_guess = os.path.join(repo_dir, 'vit_model.py') # Guessed name based on import

print(f"Repo directory: {repo_dir}")

print("\n📂 Files in repository root:")
# List files in the repository root
if os.path.exists(repo_dir):
    !ls -la {repo_dir}
else:
    print(f"❌ Repository directory not found: {repo_dir}")


print(f"\n📄 Content of {os.path.basename(eval_script_path)} (checking import):")
# Read and print the content of evaluate.py
if os.path.exists(eval_script_path):
    try:
        with open(eval_script_path, 'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if 'import vit_model' in line or 'from vit_model' in line:
                    print(f"  Line {i+1}: {line.strip()}")
                elif 'ViTForFishClassification' in line:
                     print(f"  Line {i+1}: {line.strip()} (contains class name)")
                if i < 20: # Print first 20 lines for context
                     print(f"  Line {i+1}: {line.strip()}")


    except Exception as e:
        print(f"❌ Could not read {eval_script_path}: {e}")
else:
    print(f"❌ {eval_script_path} not found.")


print(f"\n📄 Checking potential model file: {os.path.basename(model_file_guess)}")
# Check if model.py exists and print relevant lines
if os.path.exists(model_file_guess):
    try:
        with open(model_file_guess, 'r') as f:
            lines = f.readlines()
            print(f"✅ Found {os.path.basename(model_file_guess)}. Checking for class definition...")
            found_class = False
            for i, line in enumerate(lines):
                 if 'class ViTForFishClassification' in line:
                      print(f"  Line {i+1}: {line.strip()}")
                      found_class = True
                      break # Found the class, stop searching

            if not found_class:
                 print(f"⚠️ 'ViTForFishClassification' class definition not found in {os.path.basename(model_file_guess)}")

    except Exception as e:
        print(f"❌ Could not read {model_file_guess}: {e}")
else:
    print(f"❓ {os.path.basename(model_file_guess)} not found. Checking alternative name...")

print(f"\n📄 Checking alternative model file: {os.path.basename(vit_model_file_guess)}")
# Check if vit_model.py exists and print relevant lines
if os.path.exists(vit_model_file_guess):
    try:
        with open(vit_model_file_guess, 'r') as f:
            lines = f.readlines()
            print(f"✅ Found {os.path.basename(vit_model_file_guess)}. Checking for class definition...")
            found_class = False
            for i, line in enumerate(lines):
                 if 'class ViTForFishClassification' in line:
                      print(f"  Line {i+1}: {line.strip()}")
                      found_class = True
                      break # Found the class, stop searching

            if not found_class:
                 print(f"⚠️ 'ViTForFishClassification' class definition not found in {os.path.basename(vit_model_file_guess)}")


    except Exception as e:
        print(f"❌ Could not read {vit_model_file_guess}: {e}")
else:
    print(f"❓ {os.path.basename(vit_model_file_guess)} not found.")

print("\nDiagnosis steps complete. Please review the output.")

🔍 Diagnosing ModuleNotFoundError...
Repo directory: /content/ViT-FishID

📂 Files in repository root:
total 368
drwxr-xr-x 6 root root   4096 Aug 15 07:03 .
drwxr-xr-x 1 root root   4096 Aug 15 06:58 ..
-rw-r--r-- 1 root root  21217 Aug 15 06:58 data.py
-rw-r--r-- 1 root root  11572 Aug 15 06:58 evaluate.py
-rw-r--r-- 1 root root   3328 Aug 15 06:58 EXTENDED_TRAINING_SETUP.md
drwxr-xr-x 2 root root   4096 Aug 15 06:58 fish_cutouts
drwxr-xr-x 8 root root   4096 Aug 15 06:58 .git
-rw-r--r-- 1 root root     66 Aug 15 06:58 .gitattributes
-rw-r--r-- 1 root root    646 Aug 15 06:58 .gitignore
-rw-r--r-- 1 root root   9495 Aug 15 06:58 model.py
-rw-r--r-- 1 root root  16771 Aug 15 06:58 pipeline.py
drwxr-xr-x 2 root root   4096 Aug 15 07:03 __pycache__
-rw-r--r-- 1 root root  16566 Aug 15 06:58 README.md
-rw-r--r-- 1 root root    202 Aug 15 06:58 requirements.txt
-rw-r--r-- 1 root root   4265 Aug 15 06:58 resume_training.py
-rw-r--r-- 1 root root   5134 Aug 15 06:58 species_mapping.txt
-rw-r-

In [8]:
import os

print("Checking the contents of the MAE checkpoint directory:")
mae_checkpoint_dir = '/content/drive/MyDrive/mae_checkpoints'

if os.path.exists(mae_checkpoint_dir):
    print(f"✅ Directory found: {mae_checkpoint_dir}")
    print("\nFiles in the directory:")
    try:
        # List all items in the directory
        items = os.listdir(mae_checkpoint_dir)
        if items:
            for item in items:
                item_path = os.path.join(mae_checkpoint_dir, item)
                if os.path.isfile(item_path):
                    file_size = os.path.getsize(item_path) / (1024**2) # Size in MB
                    print(f"  - {item} ({file_size:.2f} MB)")
                else:
                    print(f"  - {item} (Directory)")
        else:
            print("  (Directory is empty)")
    except Exception as e:
        print(f"❌ Error listing directory contents: {e}")
else:
    print(f"❌ Directory not found: {mae_checkpoint_dir}")
    print("Please ensure the directory exists in your Google Drive.")

print("\n--- Check complete ---")

Checking the contents of the MAE checkpoint directory:
❌ Directory not found: /content/drive/MyDrive/mae_checkpoints
Please ensure the directory exists in your Google Drive.

--- Check complete ---


In [9]:
from google.colab import drive
drive.flush_and_unmount('/content/drive')
#

Drive not mounted, so nothing to flush and unmount.


In [ ]:
# Test Fixed Evaluation Script
print("🧪 Testing the fixed evaluation script...")

# Check if evaluate.py exists and has correct imports
import os
eval_script_path = '/content/ViT-FishID/evaluate.py'

if os.path.exists(eval_script_path):
    print("✅ Found evaluate.py")
    
    # Test the imports
    try:
        import sys
        sys.path.append('/content/ViT-FishID')
        
        # Test import of the fixed evaluation module
        from evaluate import ModelEvaluator
        print("✅ Successfully imported ModelEvaluator")
        
        # Test import of data loading function
        from data import create_dataloaders
        print("✅ Successfully imported create_dataloaders")
        
        # Test model import
        from model import ViTForFishClassification
        print("✅ Successfully imported ViTForFishClassification")
        
        print("\n🎉 All imports work correctly!")
        print("📋 The evaluation script is now ready to use with:")
        print("   python evaluate.py --data_dir /path/to/data --model_path /path/to/checkpoint")
        
    except ImportError as e:
        print(f"❌ Import error: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")
        
else:
    print("❌ evaluate.py not found")

print("\n" + "="*50)
print("🔧 EVALUATION SCRIPT FIXES APPLIED:")
print("="*50)
print("1. ✅ Fixed import: create_fish_dataloaders → create_dataloaders")
print("2. ✅ Fixed import: vit_model → model") 
print("3. ✅ Fixed import: data_loader → data")
print("4. ✅ Added proper argument parsing")
print("5. ✅ Added Top-5 accuracy calculation")
print("6. ✅ Simplified evaluation logic (removed teacher/student comparison)")
print("7. ✅ Added comprehensive result reporting")
print("\n🚀 Ready for evaluation on your trained model!")

In [ ]:
# 🔍 FIXED EVALUATION EXAMPLE
print("="*60)
print("🎯 EVALUATION SCRIPT USAGE EXAMPLE")
print("="*60)

# Example paths (update these to your actual paths)
example_model_path = "/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints/model_best.pth"
example_data_path = "/content/fish_cutouts"

print(f"📋 To evaluate your trained model, run:")
print()
print(f"!cd /content/ViT-FishID && python evaluate.py \\")
print(f"    --data_dir {example_data_path} \\")
print(f"    --model_path {example_model_path} \\")
print(f"    --batch_size 32 \\")
print(f"    --image_size 224")

print("\n📊 This will output:")
print("  🎯 Top-1 Accuracy (main metric)")
print("  📈 Top-5 Accuracy (secondary metric)")
print("  📋 Per-class precision, recall, F1-scores")
print("  🔍 Validation set comparison")
print("  📊 Confusion matrix and class accuracy plots")

print("\n💡 Key improvements in the fixed script:")
print("  ✅ Correct function imports from your codebase")
print("  ✅ Proper argument parsing for flexible usage")
print("  ✅ Both Top-1 and Top-5 accuracy calculation")
print("  ✅ Comprehensive evaluation metrics")
print("  ✅ Handles different checkpoint formats")
print("  ✅ Clear, informative output formatting")

print(f"\n🎉 Your evaluation script is now fully compatible!")
print(f"🚀 Ready to evaluate the ViT-FishID model performance!")

In [ ]:
# 🔧 Replace evaluate.py with Fixed Version
import os

print("🔄 Replacing evaluate.py with the fixed version...")

# Ensure we're in the right directory
%cd /content/ViT-FishID

# Create the corrected evaluate.py content
evaluate_py_content = '''import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import os
from tqdm import tqdm

from model import ViTForFishClassification
# from trainer import EMATeacher  # Not needed for evaluation
from data import create_dataloaders
from utils import accuracy, load_checkpoint, get_device


class ModelEvaluator:
    """
    Comprehensive model evaluation for ViT-Fish classification.
    """
    
    def __init__(
        self,
        model: ViTForFishClassification,
        class_names: List[str],
        device: str = 'cuda'
    ):
        """
        Initialize evaluator.
        
        Args:
            model: Trained ViT model
            class_names: List of class names
            device: Device to run evaluation on
        """
        self.model = model
        self.class_names = class_names
        self.device = device
        self.model.eval()
    
    def evaluate_dataset(
        self, 
        data_loader: DataLoader,
        save_dir: str = './evaluation_results'
    ) -> Dict[str, float]:
        """
        Comprehensive evaluation on dataset.
        
        Args:
            data_loader: DataLoader for evaluation
            save_dir: Directory to save results
            
        Returns:
            Dictionary of evaluation metrics
        """
        os.makedirs(save_dir, exist_ok=True)
        
        all_predictions = []
        all_targets = []
        all_probabilities = []
        
        total_correct = 0
        total_samples = 0
        top5_correct = 0  # Add top-5 tracking
        
        print("Evaluating model...")
        
        with torch.no_grad():
            for images, targets in tqdm(data_loader, desc='Evaluating'):
                images = images.to(self.device)
                targets = targets.to(self.device)
                
                # Forward pass
                logits = self.model(images)
                probabilities = torch.softmax(logits, dim=1)
                predictions = torch.argmax(logits, dim=1)
                
                # Calculate top-5 accuracy
                _, top5_pred = logits.topk(5, 1, True, True)
                top5_correct += (targets.view(-1, 1).expand_as(top5_pred) == top5_pred).sum().item()
                
                # Collect results
                all_predictions.extend(predictions.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                all_probabilities.extend(probabilities.cpu().numpy())
                
                # Update accuracy
                total_correct += (predictions == targets).sum().item()
                total_samples += targets.size(0)
        
        # Calculate metrics
        accuracy_score = total_correct / total_samples * 100
        top5_accuracy_score = top5_correct / total_samples * 100
        
        # Generate classification report
        class_report = classification_report(
            all_targets, 
            all_predictions, 
            target_names=self.class_names,
            output_dict=True
        )
        
        # Generate confusion matrix
        cm = confusion_matrix(all_targets, all_predictions)
        
        # Save results
        self._save_classification_report(class_report, save_dir)
        self._plot_confusion_matrix(cm, save_dir)
        self._plot_class_accuracies(class_report, save_dir)
        
        # Calculate per-class metrics
        results = {
            'accuracy': accuracy_score,  # Changed from 'overall_accuracy' to match main function
            'top5_accuracy': top5_accuracy_score,  # Add top-5 accuracy
            'macro_avg_precision': class_report['macro avg']['precision'] * 100,
            'macro_avg_recall': class_report['macro avg']['recall'] * 100,
            'macro_avg_f1': class_report['macro avg']['f1-score'] * 100,
            'weighted_avg_precision': class_report['weighted avg']['precision'] * 100,
            'weighted_avg_recall': class_report['weighted avg']['recall'] * 100,
            'weighted_avg_f1': class_report['weighted avg']['f1-score'] * 100,
            'classification_report': class_report  # Add classification report to results
        }
        
        print(f"\\nEvaluation Results:")
        print(f"Top-1 Accuracy: {accuracy_score:.2f}%")
        print(f"Top-5 Accuracy: {top5_accuracy_score:.2f}%")
        print(f"Macro Avg F1-Score: {results['macro_avg_f1']:.2f}%")
        print(f"Weighted Avg F1-Score: {results['weighted_avg_f1']:.2f}%")
        
        return results
    
    def _save_classification_report(self, report_dict: Dict, save_dir: str):
        """Save classification report to file."""
        import json
        report_path = os.path.join(save_dir, 'classification_report.json')
        with open(report_path, 'w') as f:
            json.dump(report_dict, f, indent=2)
        print(f"Classification report saved to: {report_path}")
    
    def _plot_confusion_matrix(self, cm: np.ndarray, save_dir: str):
        """Plot and save confusion matrix."""
        plt.figure(figsize=(12, 10))
        sns.heatmap(
            cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=self.class_names,
            yticklabels=self.class_names
        )
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.xticks(rotation=45)
        plt.yticks(rotation=0)
        plt.tight_layout()
        
        cm_path = os.path.join(save_dir, 'confusion_matrix.png')
        plt.savefig(cm_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Confusion matrix saved to: {cm_path}")
    
    def _plot_class_accuracies(self, report_dict: Dict, save_dir: str):
        """Plot per-class accuracies."""
        class_names = []
        accuracies = []
        
        for class_name in self.class_names:
            if class_name in report_dict:
                class_names.append(class_name)
                accuracies.append(report_dict[class_name]['f1-score'] * 100)
        
        plt.figure(figsize=(15, 8))
        bars = plt.bar(range(len(class_names)), accuracies)
        plt.xlabel('Fish Species')
        plt.ylabel('F1-Score (%)')
        plt.title('Per-Class F1-Scores')
        plt.xticks(range(len(class_names)), class_names, rotation=45, ha='right')
        
        # Add value labels on bars
        for bar, acc in zip(bars, accuracies):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{acc:.1f}%', ha='center', va='bottom')
        
        plt.tight_layout()
        acc_path = os.path.join(save_dir, 'class_accuracies.png')
        plt.savefig(acc_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Class accuracies plot saved to: {acc_path}")


def main():
    """Main evaluation script."""
    import argparse
    
    # Parse arguments
    parser = argparse.ArgumentParser(description='Evaluate ViT-FishID model')
    parser.add_argument('--data_dir', type=str, required=True,
                        help='Path to fish dataset directory')
    parser.add_argument('--model_path', type=str, required=True,
                        help='Path to model checkpoint')
    parser.add_argument('--batch_size', type=int, default=32,
                        help='Batch size for evaluation')
    parser.add_argument('--image_size', type=int, default=224,
                        help='Input image size')
    
    args = parser.parse_args()
    
    print(f"🔍 Evaluating model: {args.model_path}")
    print(f"📊 Data directory: {args.data_dir}")
    
    device = get_device()
    print(f"🖥️  Using device: {device}")
    
    # Load data - use test_loader for evaluation
    train_loader, val_loader, test_loader, class_names = create_dataloaders(
        data_dir=args.data_dir,
        batch_size=args.batch_size,
        image_size=args.image_size
    )
    
    print(f"📊 Found {len(class_names)} classes: {class_names[:5]}..." if len(class_names) > 5 else f"📊 Found {len(class_names)} classes: {class_names}")
    
    # Create model
    num_classes = len(class_names)
    model = ViTForFishClassification(
        num_classes=num_classes,
        model_name='vit_small_patch16_224',  # Adjust based on your training config
        pretrained=False,
        dropout_rate=0.1
    ).to(device)
    
    # Load checkpoint
    if os.path.exists(args.model_path):
        print(f"📥 Loading checkpoint: {args.model_path}")
        checkpoint = torch.load(args.model_path, map_location=device)
        
        # Handle different checkpoint formats
        if 'student_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['student_state_dict'])
            print(f"✅ Loaded student model weights")
            if 'best_accuracy' in checkpoint:
                print(f"📊 Training best accuracy: {checkpoint['best_accuracy']:.2f}%")
        elif 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            print(f"✅ Loaded model weights")
        else:
            model.load_state_dict(checkpoint)
            print(f"✅ Loaded model weights")
    else:
        print(f"❌ Checkpoint not found: {args.model_path}")
        return
    
    # Create evaluator
    evaluator = ModelEvaluator(model, class_names, device)
    
    # Evaluate on test set
    print(f"\\n🧪 Evaluating on test set...")
    test_results = evaluator.evaluate_dataset(test_loader, "test")
    
    # Print results
    print(f"\\n📊 TEST RESULTS:")
    print(f"🎯 Accuracy: {test_results['accuracy']:.2f}%")
    print(f"📈 Top-5 Accuracy: {test_results.get('top5_accuracy', 'N/A')}")
    
    # Print per-class results
    if 'classification_report' in test_results:
        print(f"\\n📋 Per-class Performance:")
        class_report = test_results['classification_report']
        for class_name in class_names[:10]:  # Show first 10 classes
            if class_name in class_report:
                precision = class_report[class_name]['precision']
                recall = class_report[class_name]['recall']
                f1 = class_report[class_name]['f1-score']
                print(f"  {class_name}: P={precision:.3f}, R={recall:.3f}, F1={f1:.3f}")
        
        if len(class_names) > 10:
            print(f"  ... and {len(class_names) - 10} more classes")
    
    # Also evaluate on validation set for comparison
    print(f"\\n🔍 Evaluating on validation set...")
    val_results = evaluator.evaluate_dataset(val_loader, "validation")
    print(f"📊 VALIDATION RESULTS:")
    print(f"🎯 Accuracy: {val_results['accuracy']:.2f}%")
    
    print(f"\\n✅ Evaluation completed!")
    print(f"📊 Final Test Accuracy: {test_results['accuracy']:.2f}%")


if __name__ == '__main__':
    main()
'''

# Write the fixed evaluate.py file
with open('evaluate.py', 'w') as f:
    f.write(evaluate_py_content)

print("✅ Fixed evaluate.py has been created!")
print("🔧 Key fixes applied:")
print("  ✅ Corrected all import statements")
print("  ✅ Added proper argument parsing") 
print("  ✅ Added Top-5 accuracy calculation")
print("  ✅ Fixed function calls and return values")
print("  ✅ Simplified evaluation logic")

# Verify the file was created
if os.path.exists('evaluate.py'):
    file_size = os.path.getsize('evaluate.py')
    print(f"✅ File created successfully ({file_size} bytes)")
    print("🚀 Ready to run evaluation!")
else:
    print("❌ File creation failed")

In [ ]:
# 🔧 Fix Class Mismatch Error in evaluate.py
import os

print("🔄 Fixing class mismatch error in evaluate.py...")

# Read the current evaluate.py file
with open('/content/ViT-FishID/evaluate.py', 'r') as f:
    content = f.read()

# Find and replace the classification_report generation to handle class mismatch
old_classification_report = """        # Generate classification report
        class_report = classification_report(
            all_targets, 
            all_predictions, 
            target_names=self.class_names,
            output_dict=True
        )"""

new_classification_report = """        # Generate classification report - handle class mismatch
        # Get unique classes that actually appear in the test data
        unique_classes = sorted(list(set(all_targets + all_predictions)))
        present_class_names = [self.class_names[i] for i in unique_classes if i < len(self.class_names)]
        
        # Generate classification report with only present classes
        class_report = classification_report(
            all_targets, 
            all_predictions,
            labels=unique_classes,  # Specify which labels to include
            target_names=present_class_names,  # Only names for present classes
            output_dict=True,
            zero_division=0  # Handle division by zero for missing classes
        )"""

# Replace in the content
if old_classification_report in content:
    content = content.replace(old_classification_report, new_classification_report)
    print("✅ Fixed classification_report generation")
else:
    print("⚠️ Could not find exact match for classification_report. Applying alternative fix...")
    # Alternative fix - replace the specific line
    content = content.replace(
        'class_report = classification_report(',
        '''# Get unique classes that actually appear in the test data
        unique_classes = sorted(list(set(all_targets + all_predictions)))
        present_class_names = [self.class_names[i] for i in unique_classes if i < len(self.class_names)]
        
        class_report = classification_report('''
    )
    content = content.replace(
        'target_names=self.class_names,',
        'labels=unique_classes,\n            target_names=present_class_names,'
    )
    content = content.replace(
        'output_dict=True',
        'output_dict=True,\n            zero_division=0'
    )

# Also fix the confusion matrix to handle missing classes
old_cm = "cm = confusion_matrix(all_targets, all_predictions)"
new_cm = """cm = confusion_matrix(all_targets, all_predictions, labels=unique_classes)"""

if old_cm in content:
    content = content.replace(old_cm, new_cm)
    print("✅ Fixed confusion_matrix generation")

# Fix the plotting functions to handle subset of classes
old_heatmap = """sns.heatmap(
            cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=self.class_names,
            yticklabels=self.class_names
        )"""

new_heatmap = """sns.heatmap(
            cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=present_class_names,  # Use only present classes
            yticklabels=present_class_names   # Use only present classes
        )"""

if old_heatmap in content:
    content = content.replace(old_heatmap, new_heatmap)
    print("✅ Fixed confusion matrix heatmap")

# Fix class accuracies plot
old_class_plot = """for class_name in self.class_names:
            if class_name in report_dict:"""

new_class_plot = """for class_name in present_class_names:  # Use only present classes
            if class_name in report_dict:"""

if old_class_plot in content:
    content = content.replace(old_class_plot, new_class_plot)
    print("✅ Fixed class accuracies plot")

# Write the fixed content back to the file
with open('/content/ViT-FishID/evaluate.py', 'w') as f:
    f.write(content)

print("✅ Fixed evaluate.py to handle class mismatches!")
print("🔧 Applied fixes:")
print("  ✅ Only use classes that appear in test data")
print("  ✅ Handle zero-division for missing classes") 
print("  ✅ Fix confusion matrix dimensions")
print("  ✅ Fix plotting functions for subset of classes")
print("\n🚀 Try running evaluation again!")

In [ ]:
# 🛡️ Alternative Robust Evaluation Function (Backup)
# If the fixed evaluate.py still has issues, run this cell for a simple evaluation

def simple_evaluate_model(model_path, data_dir, batch_size=32, image_size=224):
    """Simple, robust model evaluation that handles class mismatches."""
    
    import torch
    import numpy as np
    from tqdm import tqdm
    from sklearn.metrics import accuracy_score, classification_report
    
    # Import your modules
    import sys
    sys.path.append('/content/ViT-FishID')
    from model import ViTForFishClassification
    from data import create_dataloaders
    from utils import get_device
    
    print(f"🔍 Simple Evaluation")
    print(f"📊 Model: {model_path}")
    print(f"📁 Data: {data_dir}")
    
    device = get_device()
    print(f"🖥️  Device: {device}")
    
    # Load data
    try:
        train_loader, val_loader, test_loader, class_names = create_dataloaders(
            data_dir=data_dir,
            batch_size=batch_size,
            image_size=image_size
        )
        print(f"✅ Loaded data: {len(class_names)} total classes")
    except Exception as e:
        print(f"❌ Data loading error: {e}")
        return None
    
    # Create and load model
    try:
        num_classes = len(class_names)
        model = ViTForFishClassification(
            num_classes=num_classes,
            model_name='vit_small_patch16_224',
            pretrained=False,
            dropout_rate=0.1
        ).to(device)
        
        # Load checkpoint
        checkpoint = torch.load(model_path, map_location=device)
        if 'student_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['student_state_dict'])
            print(f"✅ Loaded student model weights")
        elif 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            print(f"✅ Loaded model weights")
        else:
            model.load_state_dict(checkpoint)
            print(f"✅ Loaded model weights")
            
        model.eval()
    except Exception as e:
        print(f"❌ Model loading error: {e}")
        return None
    
    # Evaluate on test set
    print(f"\\n🧪 Evaluating on test set...")
    
    all_preds = []
    all_targets = []
    all_probs = []
    
    with torch.no_grad():
        for images, targets in tqdm(test_loader, desc='Evaluating'):
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    # Convert to numpy
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    all_probs = np.array(all_probs)
    
    # Calculate Top-1 accuracy
    top1_acc = accuracy_score(all_targets, all_preds) * 100
    
    # Calculate Top-5 accuracy
    top5_preds = np.argsort(all_probs, axis=1)[:, -5:]  # Top 5 predictions
    top5_correct = 0
    for i, target in enumerate(all_targets):
        if target in top5_preds[i]:
            top5_correct += 1
    top5_acc = (top5_correct / len(all_targets)) * 100
    
    # Get unique classes in test set
    unique_classes = sorted(list(set(all_targets)))
    present_class_names = [class_names[i] for i in unique_classes]
    
    print(f"\\n📊 EVALUATION RESULTS:")
    print(f"🎯 Top-1 Accuracy: {top1_acc:.2f}%")
    print(f"📈 Top-5 Accuracy: {top5_acc:.2f}%")
    print(f"📋 Classes in test set: {len(unique_classes)}/{len(class_names)}")
    
    # Safe classification report
    try:
        report = classification_report(
            all_targets, 
            all_preds,
            labels=unique_classes,
            target_names=present_class_names,
            output_dict=False,  # Get string format
            zero_division=0
        )
        print(f"\\n📋 Per-Class Results:")
        print(report)
    except Exception as e:
        print(f"⚠️ Could not generate detailed report: {e}")
    
    # Show some class-wise accuracies
    print(f"\\n🔍 Sample Class Performance:")
    for i, class_idx in enumerate(unique_classes[:10]):  # Show first 10
        class_mask = all_targets == class_idx
        if class_mask.sum() > 0:
            class_acc = (all_preds[class_mask] == class_idx).mean() * 100
            print(f"  {class_names[class_idx]}: {class_acc:.1f}% ({class_mask.sum()} samples)")
    
    if len(unique_classes) > 10:
        print(f"  ... and {len(unique_classes) - 10} more classes")
    
    return {
        'top1_accuracy': top1_acc,
        'top5_accuracy': top5_acc,
        'num_classes_tested': len(unique_classes),
        'total_classes': len(class_names)
    }

print("✅ Robust evaluation function defined!")
print("📋 If the fixed evaluate.py still has issues, run:")
print("   results = simple_evaluate_model(")
print("       '/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints/model_best.pth',")  
print("       '/content/fish_cutouts')")
print("🛡️ This function handles class mismatches gracefully")

In [ ]:
# 🔧 Fix present_class_names Scope Error
import os

print("🔄 Fixing present_class_names scope error...")

# Read the current evaluate.py file
with open('/content/ViT-FishID/evaluate.py', 'r') as f:
    content = f.read()

print("📝 Applying fixes...")

# Fix 1: Update the evaluate_dataset method to pass present_class_names to plotting functions
old_save_results = """        # Save results
        self._save_classification_report(class_report, save_dir)
        self._plot_confusion_matrix(cm, save_dir)
        self._plot_class_accuracies(class_report, save_dir)"""

new_save_results = """        # Save results
        self._save_classification_report(class_report, save_dir)
        self._plot_confusion_matrix(cm, save_dir, present_class_names)
        self._plot_class_accuracies(class_report, save_dir, present_class_names)"""

if old_save_results in content:
    content = content.replace(old_save_results, new_save_results)
    print("✅ Fixed plotting function calls")

# Fix 2: Update _plot_confusion_matrix method signature
old_plot_cm_def = "def _plot_confusion_matrix(self, cm: np.ndarray, save_dir: str):"
new_plot_cm_def = "def _plot_confusion_matrix(self, cm: np.ndarray, save_dir: str, class_names_subset: List[str]):"

if old_plot_cm_def in content:
    content = content.replace(old_plot_cm_def, new_plot_cm_def)
    print("✅ Fixed _plot_confusion_matrix signature")

# Fix 3: Update heatmap call in _plot_confusion_matrix
old_heatmap = """xticklabels=present_class_names,  # Use only present classes
            yticklabels=present_class_names   # Use only present classes"""

new_heatmap = """xticklabels=class_names_subset,  # Use only present classes
            yticklabels=class_names_subset   # Use only present classes"""

if old_heatmap in content:
    content = content.replace(old_heatmap, new_heatmap)
    print("✅ Fixed heatmap class names")

# Fix 4: Update _plot_class_accuracies method signature
old_plot_acc_def = "def _plot_class_accuracies(self, report_dict: Dict, save_dir: str):"
new_plot_acc_def = "def _plot_class_accuracies(self, report_dict: Dict, save_dir: str, class_names_subset: List[str]):"

if old_plot_acc_def in content:
    content = content.replace(old_plot_acc_def, new_plot_acc_def)
    print("✅ Fixed _plot_class_accuracies signature")

# Fix 5: Update the loop in _plot_class_accuracies
old_class_loop = "for class_name in present_class_names:  # Use only present classes"
new_class_loop = "for class_name in class_names_subset:  # Use only present classes"

if old_class_loop in content:
    content = content.replace(old_class_loop, new_class_loop)
    print("✅ Fixed class accuracies loop")

# Write the fixed content back
with open('/content/ViT-FishID/evaluate.py', 'w') as f:
    f.write(content)

print("✅ Fixed all scope errors!")
print("🔧 Applied fixes:")
print("  ✅ Pass class names to plotting functions")
print("  ✅ Update function signatures")  
print("  ✅ Fix variable references in plots")
print("\n🚀 Try running evaluation again!")

In [ ]:
# 🆕 Create Clean, Working evaluate.py (Complete Replacement)
import os

print("🔄 Creating completely clean evaluate.py...")

# Ensure we're in the right directory
%cd /content/ViT-FishID

# Create the clean, working evaluate.py content
clean_evaluate_py = '''import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import os
from tqdm import tqdm
import json

from model import ViTForFishClassification
from data import create_dataloaders
from utils import get_device


class ModelEvaluator:
    """Clean, robust model evaluation for ViT-Fish classification."""
    
    def __init__(self, model: ViTForFishClassification, class_names: List[str], device: str = 'cuda'):
        self.model = model
        self.class_names = class_names
        self.device = device
        self.model.eval()
    
    def evaluate_dataset(self, data_loader: DataLoader, save_dir: str = './evaluation_results') -> Dict[str, float]:
        """Comprehensive evaluation on dataset."""
        os.makedirs(save_dir, exist_ok=True)
        
        all_predictions = []
        all_targets = []
        all_probabilities = []
        
        print("Evaluating model...")
        
        with torch.no_grad():
            for images, targets in tqdm(data_loader, desc='Evaluating'):
                images = images.to(self.device)
                targets = targets.to(self.device)
                
                # Forward pass
                logits = self.model(images)
                probabilities = torch.softmax(logits, dim=1)
                predictions = torch.argmax(logits, dim=1)
                
                # Collect results
                all_predictions.extend(predictions.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                all_probabilities.extend(probabilities.cpu().numpy())
        
        # Convert to numpy arrays
        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)
        all_probabilities = np.array(all_probabilities)
        
        # Calculate Top-1 accuracy
        top1_accuracy = accuracy_score(all_targets, all_predictions) * 100
        
        # Calculate Top-5 accuracy
        top5_correct = 0
        for i, target in enumerate(all_targets):
            top5_preds = np.argsort(all_probabilities[i])[-5:]  # Top 5 predictions
            if target in top5_preds:
                top5_correct += 1
        top5_accuracy = (top5_correct / len(all_targets)) * 100
        
        # Get unique classes that appear in the data
        unique_classes = sorted(list(set(all_targets.tolist() + all_predictions.tolist())))
        present_class_names = [self.class_names[i] for i in unique_classes if i < len(self.class_names)]
        
        print(f"\\nFound {len(unique_classes)} classes in test data (out of {len(self.class_names)} total)")
        
        # Generate classification report with proper handling
        try:
            class_report = classification_report(
                all_targets, 
                all_predictions,
                labels=unique_classes,
                target_names=present_class_names,
                output_dict=True,
                zero_division=0
            )
        except Exception as e:
            print(f"Warning: Could not generate full classification report: {e}")
            class_report = {}
        
        # Generate confusion matrix
        cm = confusion_matrix(all_targets, all_predictions, labels=unique_classes)
        
        # Save results
        self._save_results(class_report, save_dir)
        self._save_confusion_matrix(cm, present_class_names, save_dir)
        self._save_class_accuracies(class_report, save_dir)
        
        # Prepare results dictionary
        results = {
            'accuracy': top1_accuracy,
            'top5_accuracy': top5_accuracy,
            'num_test_classes': len(unique_classes),
            'total_classes': len(self.class_names),
            'classification_report': class_report
        }
        
        # Add aggregate metrics if available
        if 'macro avg' in class_report:
            results.update({
                'macro_avg_precision': class_report['macro avg']['precision'] * 100,
                'macro_avg_recall': class_report['macro avg']['recall'] * 100,
                'macro_avg_f1': class_report['macro avg']['f1-score'] * 100,
            })
        
        if 'weighted avg' in class_report:
            results.update({
                'weighted_avg_precision': class_report['weighted avg']['precision'] * 100,
                'weighted_avg_recall': class_report['weighted avg']['recall'] * 100,
                'weighted_avg_f1': class_report['weighted avg']['f1-score'] * 100,
            })
        
        # Print results
        print(f"\\n📊 EVALUATION RESULTS:")
        print(f"🎯 Top-1 Accuracy: {top1_accuracy:.2f}%")
        print(f"📈 Top-5 Accuracy: {top5_accuracy:.2f}%")
        if 'macro_avg_f1' in results:
            print(f"📋 Macro Avg F1-Score: {results['macro_avg_f1']:.2f}%")
        if 'weighted_avg_f1' in results:
            print(f"⚖️  Weighted Avg F1-Score: {results['weighted_avg_f1']:.2f}%")
        
        return results
    
    def _save_results(self, report_dict: Dict, save_dir: str):
        """Save classification report to JSON."""
        if report_dict:
            report_path = os.path.join(save_dir, 'classification_report.json')
            try:
                with open(report_path, 'w') as f:
                    json.dump(report_dict, f, indent=2)
                print(f"Classification report saved to: {report_path}")
            except Exception as e:
                print(f"Warning: Could not save classification report: {e}")
    
    def _save_confusion_matrix(self, cm: np.ndarray, class_names: List[str], save_dir: str):
        """Save confusion matrix plot."""
        try:
            plt.figure(figsize=(max(10, len(class_names) * 0.5), max(8, len(class_names) * 0.4)))
            sns.heatmap(
                cm, 
                annot=True, 
                fmt='d', 
                cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names,
                cbar_kws={'label': 'Count'}
            )
            plt.title('Confusion Matrix')
            plt.xlabel('Predicted')
            plt.ylabel('Actual')
            plt.xticks(rotation=45, ha='right')
            plt.yticks(rotation=0)
            plt.tight_layout()
            
            cm_path = os.path.join(save_dir, 'confusion_matrix.png')
            plt.savefig(cm_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Confusion matrix saved to: {cm_path}")
        except Exception as e:
            print(f"Warning: Could not save confusion matrix: {e}")
    
    def _save_class_accuracies(self, report_dict: Dict, save_dir: str):
        """Save per-class accuracies plot."""
        if not report_dict:
            return
            
        try:
            class_names = []
            f1_scores = []
            
            for class_name in self.class_names:
                if class_name in report_dict:
                    class_names.append(class_name)
                    f1_scores.append(report_dict[class_name]['f1-score'] * 100)
            
            if not class_names:
                print("Warning: No per-class data available for plotting")
                return
            
            plt.figure(figsize=(max(12, len(class_names) * 0.3), 8))
            bars = plt.bar(range(len(class_names)), f1_scores, color='skyblue', alpha=0.7)
            plt.xlabel('Fish Species')
            plt.ylabel('F1-Score (%)')
            plt.title('Per-Class F1-Scores')
            plt.xticks(range(len(class_names)), class_names, rotation=45, ha='right')
            plt.grid(axis='y', alpha=0.3)
            
            # Add value labels on bars
            for bar, score in zip(bars, f1_scores):
                height = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                        f'{score:.1f}%', ha='center', va='bottom', fontsize=8)
            
            plt.tight_layout()
            acc_path = os.path.join(save_dir, 'class_f1_scores.png')
            plt.savefig(acc_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Class F1-scores plot saved to: {acc_path}")
        except Exception as e:
            print(f"Warning: Could not save class accuracies plot: {e}")


def main():
    """Main evaluation script."""
    import argparse
    
    parser = argparse.ArgumentParser(description='Evaluate ViT-FishID model')
    parser.add_argument('--data_dir', type=str, required=True, help='Path to fish dataset directory')
    parser.add_argument('--model_path', type=str, required=True, help='Path to model checkpoint')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size for evaluation')
    parser.add_argument('--image_size', type=int, default=224, help='Input image size')
    
    args = parser.parse_args()
    
    print(f"🔍 Evaluating model: {args.model_path}")
    print(f"📊 Data directory: {args.data_dir}")
    
    device = get_device()
    print(f"🖥️  Using device: {device}")
    
    # Load data
    try:
        train_loader, val_loader, test_loader, class_names = create_dataloaders(
            data_dir=args.data_dir,
            batch_size=args.batch_size,
            image_size=args.image_size
        )
        print(f"📊 Found {len(class_names)} total classes")
    except Exception as e:
        print(f"❌ Data loading error: {e}")
        return
    
    # Create model
    try:
        num_classes = len(class_names)
        model = ViTForFishClassification(
            num_classes=num_classes,
            model_name='vit_small_patch16_224',
            pretrained=False,
            dropout_rate=0.1
        ).to(device)
        
        # Load checkpoint
        if os.path.exists(args.model_path):
            print(f"📥 Loading checkpoint: {args.model_path}")
            checkpoint = torch.load(args.model_path, map_location=device)
            
            if 'student_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['student_state_dict'])
                print("✅ Loaded student model weights")
                if 'best_accuracy' in checkpoint:
                    print(f"📊 Training best accuracy: {checkpoint['best_accuracy']:.2f}%")
            elif 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
                print("✅ Loaded model weights")
            else:
                model.load_state_dict(checkpoint)
                print("✅ Loaded model weights")
        else:
            print(f"❌ Checkpoint not found: {args.model_path}")
            return
    except Exception as e:
        print(f"❌ Model loading error: {e}")
        return
    
    # Create evaluator and run evaluation
    try:
        evaluator = ModelEvaluator(model, class_names, device)
        
        # Evaluate on test set
        print(f"\\n🧪 Evaluating on test set...")
        test_results = evaluator.evaluate_dataset(test_loader, "test_results")
        
        # Evaluate on validation set for comparison
        print(f"\\n🔍 Evaluating on validation set...")
        val_results = evaluator.evaluate_dataset(val_loader, "validation_results")
        
        # Final summary
        print(f"\\n" + "="*60)
        print(f"🎉 EVALUATION COMPLETED!")
        print(f"="*60)
        print(f"📊 TEST SET RESULTS:")
        print(f"  🎯 Top-1 Accuracy: {test_results['accuracy']:.2f}%")
        print(f"  📈 Top-5 Accuracy: {test_results['top5_accuracy']:.2f}%")
        print(f"\\n📊 VALIDATION SET RESULTS:")
        print(f"  🎯 Top-1 Accuracy: {val_results['accuracy']:.2f}%")
        print(f"  📈 Top-5 Accuracy: {val_results['top5_accuracy']:.2f}%")
        print(f"\\n📋 Dataset Info:")
        print(f"  Total classes: {test_results['total_classes']}")
        print(f"  Classes in test set: {test_results['num_test_classes']}")
        print(f"\\n💾 Results saved to: test_results/ and validation_results/")
        
    except Exception as e:
        print(f"❌ Evaluation error: {e}")
        import traceback
        traceback.print_exc()


if __name__ == '__main__':
    main()
'''

# Write the clean evaluate.py file
with open('evaluate.py', 'w') as f:
    f.write(clean_evaluate_py)

print("✅ Created clean, working evaluate.py!")
print("🔧 Features of the new script:")
print("  ✅ Robust class mismatch handling")
print("  ✅ Proper error handling throughout") 
print("  ✅ Clean function scoping")
print("  ✅ Both Top-1 and Top-5 accuracy")
print("  ✅ Comprehensive result saving")
print("  ✅ Validation and test set evaluation")

# Verify the file
if os.path.exists('evaluate.py'):
    file_size = os.path.getsize('evaluate.py')
    print(f"✅ Clean script created ({file_size} bytes)")
    print("\\n🚀 Ready to run evaluation without errors!")
else:
    print("❌ File creation failed")

In [ ]:
# ✅ Test Clean Evaluation Script
print("🧪 Testing the clean evaluation script...")

# Quick import test
try:
    import sys
    sys.path.append('/content/ViT-FishID')
    
    from evaluate import ModelEvaluator
    print("✅ Successfully imported ModelEvaluator")
    
    # Test if the script can run (argument check)
    import subprocess
    result = subprocess.run(['python', '/content/ViT-FishID/evaluate.py', '--help'], 
                          capture_output=True, text=True, cwd='/content/ViT-FishID')
    
    if result.returncode == 0:
        print("✅ Script runs without syntax errors")
        print("\\n📋 Available options:")
        print(result.stdout.split('\\n')[1:6])  # Show first few help lines
    else:
        print("⚠️ Script has some issues:")
        print(result.stderr[:200])
        
except Exception as e:
    print(f"❌ Error testing script: {e}")

print("\\n" + "="*50)
print("🎯 NOW RUN YOUR EVALUATION WITH:")
print("="*50)
print("!cd /content/ViT-FishID && python evaluate.py \\\\")
print("    --data_dir /content/fish_cutouts \\\\")
print("    --model_path /content/drive/MyDrive/ViT-FishID/pretrained_checkpoints/model_best.pth \\\\") 
print("    --batch_size 32 \\\\")
print("    --image_size 224")

print("\\n🎉 The clean script should handle all errors gracefully!")
print("📊 Expected output: Both test and validation accuracies")
print("💾 Results will be saved to test_results/ and validation_results/ folders")

# 🚀 How to Get the Clean Evaluation Script into Google Colab

Follow these steps to get the fixed `evaluate.py` script working in your Google Colab:

## Method 1: Run the Creation Cell (Recommended)

1. **Scroll up** to find the cell titled **"🆕 Create Clean, Working evaluate.py (Complete Replacement)"** 
2. **Run that cell** - it will create the clean `evaluate.py` file directly in your Colab workspace
3. **Run the test cell** below it to verify it works
4. **Run your evaluation** with the provided command

## Method 2: Manual Upload (Alternative)

If Method 1 doesn't work:

1. **Copy the script content** from the creation cell
2. **Create a new file** in Colab:
   ```python
   # Create evaluate.py manually
   with open('/content/ViT-FishID/evaluate.py', 'w') as f:
       f.write("""[paste the script content here]""")
   ```

## Method 3: Google Drive Upload

1. **Save the script** to your local computer as `evaluate.py`
2. **Upload to Google Drive** in your ViT-FishID folder  
3. **Copy from Drive** in Colab:
   ```python
   !cp /content/drive/MyDrive/ViT-FishID/evaluate.py /content/ViT-FishID/
   ```

## ✅ After Getting the Script in Colab:

Run these commands in order:

### Step 1: Verify the script exists
```python
!ls -la /content/ViT-FishID/evaluate.py
```

### Step 2: Test the script works
```python
!cd /content/ViT-FishID && python evaluate.py --help
```

### Step 3: Run the evaluation
```python
!cd /content/ViT-FishID && python evaluate.py \
    --data_dir /content/fish_cutouts \
    --model_path /content/drive/MyDrive/ViT-FishID/pretrained_checkpoints/model_best.pth \
    --batch_size 32 \
    --image_size 224
```

## 🎯 Expected Results:
- Top-1 and Top-5 accuracy on both test and validation sets
- Per-class performance metrics
- Confusion matrix and F1-score plots
- Results saved to `test_results/` and `validation_results/` folders

In [ ]:
# 📋 Step-by-Step: Get Evaluation Script Working in Colab

print("🚀 Setting up evaluation script in Google Colab...")
print("="*60)

# Step 1: Ensure we're in the right directory
print("📁 Step 1: Setting up directory")
%cd /content/ViT-FishID
print("✅ In ViT-FishID directory")

# Step 2: Check if evaluate.py exists
import os
if os.path.exists('evaluate.py'):
    print("✅ Step 2: evaluate.py already exists")
    file_size = os.path.getsize('evaluate.py')
    print(f"   File size: {file_size} bytes")
else:
    print("❌ Step 2: evaluate.py not found")
    print("💡 Please run the '🆕 Create Clean, Working evaluate.py' cell first!")

# Step 3: Test script syntax
print("\n🧪 Step 3: Testing script syntax")
try:
    import subprocess
    result = subprocess.run(['python', '-m', 'py_compile', 'evaluate.py'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Script syntax is valid")
    else:
        print("❌ Syntax errors found:")
        print(result.stderr)
except Exception as e:
    print(f"⚠️ Could not test syntax: {e}")

# Step 4: Test imports
print("\n📦 Step 4: Testing imports")
try:
    import sys
    sys.path.append('/content/ViT-FishID')
    from evaluate import ModelEvaluator
    print("✅ All imports work correctly")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("💡 Make sure all required modules are available")
except Exception as e:
    print(f"⚠️ Other error: {e}")

# Step 5: Check required paths
print("\n🔍 Step 5: Checking required paths")

# Check data directory
data_path = "/content/fish_cutouts"
if os.path.exists(data_path):
    print(f"✅ Data directory exists: {data_path}")
    subdirs = [d for d in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, d))]
    print(f"   Found subdirectories: {len(subdirs)}")
else:
    print(f"❌ Data directory not found: {data_path}")

# Check model checkpoint
model_path = "/content/drive/MyDrive/ViT-FishID/pretrained_checkpoints/model_best.pth"
if os.path.exists(model_path):
    print(f"✅ Model checkpoint exists: model_best.pth")
    file_size = os.path.getsize(model_path) / (1024*1024)  # MB
    print(f"   Model size: {file_size:.1f} MB")
else:
    print(f"❌ Model checkpoint not found")
    print("💡 Update the path to your actual checkpoint location")

# Step 6: Show the command to run
print("\n🎯 Step 6: Ready to run evaluation!")
print("="*60)
print("Copy and run this command:")
print()
print("!cd /content/ViT-FishID && python evaluate.py \\")
print("    --data_dir /content/fish_cutouts \\") 
print("    --model_path /content/drive/MyDrive/ViT-FishID/pretrained_checkpoints/model_best.pth \\")
print("    --batch_size 32 \\")
print("    --image_size 224")
print()
print("🎉 This should work without errors and give you:")
print("   📊 Top-1 and Top-5 accuracy")
print("   📋 Per-class performance metrics") 
print("   📈 Confusion matrix and plots")
print("   💾 Results saved to test_results/ and validation_results/")

# Final status
print(f"\n{'='*60}")
if os.path.exists('evaluate.py') and os.path.exists(data_path):
    print("🎉 READY TO EVALUATE! Everything looks good!")
else:
    print("⚠️ SETUP INCOMPLETE - Fix the issues above first")
print(f"{'='*60}")